# Economic Indicator Cleaning, Validation, and Merging

This notebook cleans, validates, and merges the economic indicators for our analysis into three dataframes according to the frequency with which the data is collected (daily, monthly, and quarterly). Some economic indicators have been transformed for inclusion in dataframes that do not correspond to the frequency with which they are collected. For example, semi-monthly PMI values have been averaged by month. Economic indicators were selected on the basis of being mentioned in the project brief, in meetings with the Bank of England, in the House of Commons Library research briefing entitled [Economic Indicators: Key statistics for the UK economy](https://commonslibrary.parliament.uk/research-briefings/cbp-9040/), and in the scholarly literature we reviewed to prepare our research design. Whenever possible we collected data from official sources such as the [Office for National Statistics](https://www.ons.gov.uk/) (ONS).

## Import Libraries

In [1]:
# Import relevant libraries.
import numpy as np
import os
import pandas as pd
import re
from datetime import datetime
from functools import reduce
from datetime import datetime, timedelta
from pandas.tseries.offsets import QuarterBegin, QuarterEnd

## Functions

### Data Validation

In [2]:
# Data validation function
def validate_data(df):
    """
    This function compiles several functions for validating a dataframe.

    Arg:
        df: dataframe name

    Returns:
        prints missing counts
        prints unique counts per column
        prints duplicate counts
        prints summary statistics for the dataframe
    """
    #check for missing values
    missing_counts = df.isnull().sum()
    print("Missing values per column:")
    print(missing_counts)
    
    #check for unique values
    unique_counts = df.nunique()
    print("\nUnique values per column:")
    print(unique_counts)
    
    #checking for duplicates
    duplicate_count = df.duplicated().sum()
    print("\nNumber of duplicate rows:")
    print(duplicate_count)
    
    #summary stats
    summary_stats = df.describe()
    print("\nSummary statistics:")
    print(summary_stats)

    #zero values
    no_zeros = df.ne(0).all()
    print("\nColumn has no zero values?")
    print(no_zeros)

### Split Dataframes with Annual, Quarterly, and Monthly values

Many datasets provided by the ONS contain monthly, quarterly, and annual values in the same column. This function, which was developed with the assistance of the Claude AI Chatbot, splits these datasets into three separate dataframes by monthly, quarterly, and annual values.

In [3]:
def split_dataframe(df, date='date', prefix=None):
    """
    Split a dataframe with mixed date formats into three separate dataframes
    and save them as CSV files in the current directory.
    
    Parameters:
    - df: Dataframe containing the mixed data
    - date: Column name containing the date values
    - prefix: String to use as prefix for the saved csv files

    Returns:
    - df_annual: Dataframe with only annual values
    - df_quarterly: Dataframe with only quarterly values
    - df_monthly: Dataframe with only monthly values
    """
    # Create masks for each type of date format
    annual_mask = df[date].apply(lambda x: bool(re.match(r'^\d{4}$', str(x))))
    quarterly_mask = df[date].apply(lambda x: bool(re.match(r'^\d{4}\s+Q[1-4]$', str(x))))
    monthly_mask = df[date].apply(lambda x: bool(re.match(r'^\d{4}\s+[A-Z]{3}$', str(x))))
    
    # Split the dataframe based on the masks
    df_annual = df[annual_mask].copy()
    df_quarterly = df[quarterly_mask].copy()
    df_monthly = df[monthly_mask].copy()

    # Convert date columns to pandas Period format
    if len(df_annual) > 0:
        # Convert annual dates to Period with annual frequency
        df_annual['date'] = df_annual['date'].apply(lambda x: pd.Period(str(x), freq='A-DEC'))
    
    if len(df_quarterly) > 0:
        # Convert quarterly dates to Period with quarterly frequency
        def quarter_to_period(q_str):
            year, quarter = q_str.split()
            quarter_num = int(quarter[1])
            return pd.Period(f"{year}Q{quarter_num}", freq='Q-DEC')
        
        df_quarterly['date'] = df_quarterly['date'].apply(quarter_to_period)
    
    if len(df_monthly) > 0:
        # Convert monthly dates to Period with monthly frequency
        def month_to_period(m_str):
            year, month = m_str.split()
            dt = pd.to_datetime(m_str, format='%Y %b')
            return pd.Period(dt, freq='M')
        
        df_monthly['date'] = df_monthly['date'].apply(month_to_period)
    
    # Calculate how many rows were assigned
    total_assigned = len(df_annual) + len(df_quarterly) + len(df_monthly)
    unassigned = len(df) - total_assigned
    
    print(f"Original dataset rows: {len(df)}")
    print(f"Annual dataset rows: {len(df_annual)}")
    print(f"Quarterly dataset rows: {len(df_quarterly)}")
    print(f"Monthly dataset rows: {len(df_monthly)}")
    
    if unassigned > 0:
        print(f"WARNING: {unassigned} rows did not match any expected date format")
        unmatched_df = df[~(annual_mask | quarterly_mask | monthly_mask)]
        print("Sample of unmatched dates:")
        print(unmatched_df[date].head())

    # Save the dataframes as CSV files if a prefix is provided
    if prefix:
        annual_file = f"{prefix}_annual_clean.csv"
        quarterly_file = f"{prefix}_quarterly_clean.csv"
        monthly_file = f"{prefix}_monthly_clean.csv"

    df_annual.to_csv(annual_file, index=False)
    df_quarterly.to_csv(quarterly_file, index=False)
    df_monthly.to_csv(monthly_file, index=False)
    
    return df_annual, df_quarterly, df_monthly

# Example usage:
# cpi_annual, cpi_quarterly, cpi_monthly = split_dataframe(cpi, 'date', prefix='cpi')

### Convert Quarterly Dataframes to Monthly

In [4]:
def period_quarterly_to_monthly(df_quarterly, date_col):
    """
    Convert a dataframe with values that are quarterly into a dataframe
    with rows for each month so it can be merged with other monthly dataframes.
    
    Parameters:
    - df: Dataframe containing the quarterly data
    - date_col: Column name containing the date values

    Returns:
    - monthly_data: Dataframe with values for each month
    """
    
    monthly_data = []
    
    for _, row in df_quarterly.iterrows():
        quarter_period = row[date_col]  # This is like "2024Q4"
        
        # Get the year and quarter number
        year = quarter_period.year
        quarter = quarter_period.quarter
        
        # Calculate the 3 months in this quarter
        start_month = (quarter - 1) * 3 + 1
        
        for month_offset in range(3):
            month_num = start_month + month_offset
            month_period = pd.Period(year=year, month=month_num, freq='M')
            
            new_row = row.copy()
            new_row['date'] = month_period
            monthly_data.append(new_row)
    
    return pd.DataFrame(monthly_data)

## Example usage:    
## Convert your quarterly data to monthly
# df_monthly = period_quarterly_to_monthly(df_quarterly, 'date_column')

## Data Import, Cleaning, and Validating

### Bank Rate

Source: https://www.bankofengland.co.uk/boeapps/database/Bank-Rate.asp

In [5]:
# Import .csv
bank_rate = pd.read_csv('bank_rate.csv')
bank_rate

,Date Changed,Rate
0,08 May 25,4.25
1,06 Feb 25,4.50
2,07 Nov 24,4.75
3,01 Aug 24,5.00
4,03 Aug 23,5.25
...,...,...
251,10 Mar 75,10.25
252,17 Feb 75,10.50
253,10 Feb 75,10.75
254,27 Jan 75,11.00


In [6]:
# Rename column headers
bank_rate.rename(columns={'Date Changed': 'date', 'Rate': 'bank_rate'}, inplace=True)
bank_rate

,date,bank_rate
0,08 May 25,4.25
1,06 Feb 25,4.50
2,07 Nov 24,4.75
3,01 Aug 24,5.00
4,03 Aug 23,5.25
...,...,...
251,10 Mar 75,10.25
252,17 Feb 75,10.50
253,10 Feb 75,10.75
254,27 Jan 75,11.00


In [7]:
# Change date to datetime data type
bank_rate['date'] = pd.to_datetime(bank_rate['date'])
bank_rate

/var/folders/hk/dv8kkk0x06n17jn62mhyd1cw0000gn/T/ipykernel_37434/2418949765.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  bank_rate['date'] = pd.to_datetime(bank_rate['date'])


,date,bank_rate
0,2025-05-08,4.25
1,2025-02-06,4.50
2,2024-11-07,4.75
3,2024-08-01,5.00
4,2023-08-03,5.25
...,...,...
251,1975-03-10,10.25
252,1975-02-17,10.50
253,1975-02-10,10.75
254,1975-01-27,11.00


In [8]:
# Create a complete date range to merge with speech dates
all_dates = pd.date_range(
    start=bank_rate['date'].min(),
    end=bank_rate['date'].max(),
    freq='D'
)

# Reindex bank rates to daily frequency and forward fill
bank_rates_daily = (bank_rate
                   .set_index('date')
                   .reindex(all_dates)
                   .ffill()
                   .reset_index()
                   .rename(columns={'index': 'date'}))
bank_rates_daily

,date,bank_rate
0,1975-01-20,11.25
1,1975-01-21,11.25
2,1975-01-22,11.25
3,1975-01-23,11.25
4,1975-01-24,11.25
...,...,...
18367,2025-05-04,4.50
18368,2025-05-05,4.50
18369,2025-05-06,4.50
18370,2025-05-07,4.50


In [9]:
validate_data(bank_rates_daily)

Missing values per column:
date         0
bank_rate    0
dtype: int64

Unique values per column:
date         18372
bank_rate      114
dtype: int64

Number of duplicate rows:
0

Summary statistics:
                      date     bank_rate
count                18372  18372.000000
mean   2000-03-14 12:00:00      6.167891
min    1975-01-20 00:00:00      0.100000
25%    1987-08-17 18:00:00      0.750000
50%    2000-03-14 12:00:00      5.500000
75%    2012-10-10 06:00:00     10.000000
max    2025-05-08 00:00:00     17.000000
std                    NaN      4.666921

Column has no zero values?
date         True
bank_rate    True
dtype: bool


**The average bank rate from 1975 until the present is 6.17 percent.**

In [10]:
# Save to csv
bank_rates_daily.to_csv('bank_rates_daily_clean.csv', index=False)

### GDP

(Quarter on Quarter growth: CVM SA %)

Source: https://www.ons.gov.uk/economy/grossdomesticproductgdp/timeseries/ihyq/qna

In [11]:
# Load .csv
gdp = pd.read_csv('gdp_rate.csv', skiprows=7)
gdp

,Important notes,Unnamed: 1
0,1955 Q2,0.0
1,1955 Q3,2.0
2,1955 Q4,-0.6
3,1956 Q1,1.1
4,1956 Q2,-0.1
...,...,...
274,2023 Q4,-0.2
275,2024 Q1,0.9
276,2024 Q2,0.5
277,2024 Q3,0.0


In [12]:
# Rename columns
gdp.rename(columns={'Important notes': 'date', 'Unnamed: 1': 'gdp_rate'}, inplace=True)
gdp

,date,gdp_rate
0,1955 Q2,0.0
1,1955 Q3,2.0
2,1955 Q4,-0.6
3,1956 Q1,1.1
4,1956 Q2,-0.1
...,...,...
274,2023 Q4,-0.2
275,2024 Q1,0.9
276,2024 Q2,0.5
277,2024 Q3,0.0


In [13]:
# Change date data type to quarterly period
gdp['date'] = pd.PeriodIndex(gdp['date'].str.replace(' ', '', regex=False), freq='Q')
gdp

,date,gdp_rate
0,1955Q2,0.0
1,1955Q3,2.0
2,1955Q4,-0.6
3,1956Q1,1.1
4,1956Q2,-0.1
...,...,...
274,2023Q4,-0.2
275,2024Q1,0.9
276,2024Q2,0.5
277,2024Q3,0.0


In [14]:
validate_data(gdp)

Missing values per column:
date        0
gdp_rate    0
dtype: int64

Unique values per column:
date        279
gdp_rate     52
dtype: int64

Number of duplicate rows:
0

Summary statistics:
         gdp_rate
count  279.000000
mean     0.579211
std      1.898333
min    -20.300000
25%      0.100000
50%      0.600000
75%      0.900000
max     16.800000

Column has no zero values?
date         True
gdp_rate    False
dtype: bool


**The average rate of quarter-over-quarter GDP growth is .58 percent. Some zero values are to be expected because GDP does not always change from one quarter to the next.**

In [15]:
# Save to csv
gdp.to_csv('gdp_clean.csv', index=False)

**Creating a monthly version of the dataframe to be included with other monthly economic indicators:**

In [16]:
# Convert quarterly data to monthly
gdp_monthly = period_quarterly_to_monthly(gdp, 'date')
gdp_monthly

,date,gdp_rate
0,1955-04,0.0
0,1955-05,0.0
0,1955-06,0.0
1,1955-07,2.0
1,1955-08,2.0
...,...,...
277,2024-08,0.0
277,2024-09,0.0
278,2024-10,0.1
278,2024-11,0.1


In [17]:
# Save to csv
gdp_monthly.to_csv('gdp_monthly_clean.csv', index=False)

### Inflation

(CPI Annual Rate)

Note: Dataset contains annual, quarterly, and monthly values

Source: https://www.ons.gov.uk/economy/inflationandpriceindices/timeseries/d7g7/mm23

In [18]:
# Load .csv
cpi = pd.read_csv('cpi_rate.csv', skiprows=7)
cpi

,Important notes,Unnamed: 1
0,1989,5.2
1,1990,7.0
2,1991,7.5
3,1992,4.2
4,1993,2.5
...,...,...
611,2024 NOV,2.6
612,2024 DEC,2.5
613,2025 JAN,3.0
614,2025 FEB,2.8


In [19]:
# Rename columns
cpi.rename(columns={'Important notes': 'date', 'Unnamed: 1': 'cpi_rate'}, inplace=True)
cpi

,date,cpi_rate
0,1989,5.2
1,1990,7.0
2,1991,7.5
3,1992,4.2
4,1993,2.5
...,...,...
611,2024 NOV,2.6
612,2024 DEC,2.5
613,2025 JAN,3.0
614,2025 FEB,2.8


In [20]:
# Split dataframe by annual, quarterly, and monthly values
cpi_annual, cpi_quarterly, cpi_monthly = split_dataframe(cpi, 'date', prefix='cpi')

Original dataset rows: 616
Annual dataset rows: 36
Quarterly dataset rows: 145
Monthly dataset rows: 435


In [21]:
cpi_annual.head()

,date,cpi_rate
0,1989,5.2
1,1990,7.0
2,1991,7.5
3,1992,4.2
4,1993,2.5


In [22]:
cpi_quarterly.head()

,date,cpi_rate
36,1989Q1,5.0
37,1989Q2,5.3
38,1989Q3,5.1
39,1989Q4,5.5
40,1990Q1,5.8


In [23]:
cpi_monthly.head()

,date,cpi_rate
181,1989-01,4.9
182,1989-02,5.0
183,1989-03,5.0
184,1989-04,5.3
185,1989-05,5.3


In [24]:
validate_data(cpi_monthly)

Missing values per column:
date        0
cpi_rate    0
dtype: int64

Unique values per column:
date        435
cpi_rate     86
dtype: int64

Number of duplicate rows:
0

Summary statistics:
         cpi_rate
count  435.000000
mean     2.820690
std      2.164211
min     -0.100000
25%      1.500000
50%      2.300000
75%      3.100000
max     11.100000

Column has no zero values?
date         True
cpi_rate    False
dtype: bool


In [25]:
validate_data(cpi_quarterly)

Missing values per column:
date        0
cpi_rate    0
dtype: int64

Unique values per column:
date        145
cpi_rate     60
dtype: int64

Number of duplicate rows:
0

Summary statistics:
         cpi_rate
count  145.000000
mean     2.816552
std      2.156161
min      0.000000
25%      1.500000
50%      2.300000
75%      3.100000
max     10.700000

Column has no zero values?
date         True
cpi_rate    False
dtype: bool


**The average rate of inflation from 1989 to the present is 2.82 percent. Some zero values are to be expected because inflation does not always change from one period to the next.**

In [26]:
# Save to csv
cpi_monthly.to_csv('cpi_monthly_clean.csv', index=False)
cpi_quarterly.to_csv('cpi_quarterly_clean.csv', index=False)

### Unemployment Rate

(Aged 16 and over, seasonally adjusted %)

Note: Dataset contains annual, quarterly, and monthly values

Source: https://www.ons.gov.uk/employmentandlabourmarket/peoplenotinwork/unemployment/timeseries/mgsx/lms

In [27]:
unemp = pd.read_csv('unemployment_rate.csv', skiprows=7)
unemp

,Important notes,Unnamed: 1
0,1971,4.1
1,1972,4.3
2,1973,3.7
3,1974,3.7
4,1975,4.5
...,...,...
913,2024 SEP,4.3
914,2024 OCT,4.4
915,2024 NOV,4.4
916,2024 DEC,4.4


In [28]:
# Rename columns
unemp.rename(columns={'Important notes': 'date', 'Unnamed: 1': 'unemp_rate'}, inplace=True)
unemp

,date,unemp_rate
0,1971,4.1
1,1972,4.3
2,1973,3.7
3,1974,3.7
4,1975,4.5
...,...,...
913,2024 SEP,4.3
914,2024 OCT,4.4
915,2024 NOV,4.4
916,2024 DEC,4.4


In [29]:
# Split dataframe by annual, quarterly, and monthly values
unemp_annual, unemp_quarterly, unemp_monthly = split_dataframe(unemp, 'date', prefix='unemp')

Original dataset rows: 918
Annual dataset rows: 54
Quarterly dataset rows: 216
Monthly dataset rows: 648


In [30]:
unemp_annual.head()

,date,unemp_rate
0,1971,4.1
1,1972,4.3
2,1973,3.7
3,1974,3.7
4,1975,4.5


In [31]:
unemp_quarterly.head()

,date,unemp_rate
54,1971Q1,3.8
55,1971Q2,4.1
56,1971Q3,4.2
57,1971Q4,4.4
58,1972Q1,4.5


In [32]:
unemp_monthly.head()

,date,unemp_rate
270,1971-02,3.8
271,1971-03,3.9
272,1971-04,4.0
273,1971-05,4.1
274,1971-06,4.1


In [33]:
validate_data(unemp_monthly)

Missing values per column:
date          0
unemp_rate    0
dtype: int64

Unique values per column:
date          648
unemp_rate     86
dtype: int64

Number of duplicate rows:
0

Summary statistics:
       unemp_rate
count  648.000000
mean     6.653858
std      2.383680
min      3.400000
25%      4.800000
50%      5.700000
75%      8.200000
max     11.900000

Column has no zero values?
date          True
unemp_rate    True
dtype: bool


In [34]:
validate_data(unemp_quarterly)

Missing values per column:
date          0
unemp_rate    0
dtype: int64

Unique values per column:
date          216
unemp_rate     73
dtype: int64

Number of duplicate rows:
0

Summary statistics:
       unemp_rate
count  216.000000
mean     6.653241
std      2.385516
min      3.400000
25%      4.800000
50%      5.700000
75%      8.200000
max     11.900000

Column has no zero values?
date          True
unemp_rate    True
dtype: bool


**The average unemployment rate from 1971 to the present is 6.65 percent.**

In [35]:
# Save to csv
unemp_monthly.to_csv('unemp_monthly_clean.csv', index=False)
unemp_quarterly.to_csv('unemp_quarterly_clean.csv', index=False)

### Wage Growth (Monthly)

(Average Weekly Earnings: Whole Economy Year on Year Three Month Average Growth (%): Seasonally Adjusted Regular Pay Excluding Arrears)

Source: https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/timeseries/kai9/lms

In [36]:
# Load .csv
wage_monthly = pd.read_csv('wage_rate.csv', skiprows=7)
wage_monthly

,Important notes,Unnamed: 1
0,2001 MAR,3.9
1,2001 APR,4.5
2,2001 MAY,4.9
3,2001 JUN,5.0
4,2001 JUL,5.1
...,...,...
283,2024 OCT,5.4
284,2024 NOV,5.6
285,2024 DEC,5.9
286,2025 JAN,5.8


In [37]:
# Rename columns
wage_monthly.rename(columns={'Important notes': 'date', 'Unnamed: 1': 'wage_rate'}, inplace=True)

In [38]:
# Convert 'date' column to datetime
wage_monthly['date'] = pd.to_datetime(wage_monthly['date'], format='%Y %b')
wage_monthly['date'] = wage_monthly['date'].dt.to_period('M')
wage_monthly

,date,wage_rate
0,2001-03,3.9
1,2001-04,4.5
2,2001-05,4.9
3,2001-06,5.0
4,2001-07,5.1
...,...,...
283,2024-10,5.4
284,2024-11,5.6
285,2024-12,5.9
286,2025-01,5.8


In [39]:
validate_data(wage_monthly)

Missing values per column:
date         0
wage_rate    0
dtype: int64

Unique values per column:
date         288
wage_rate     68
dtype: int64

Number of duplicate rows:
0

Summary statistics:
        wage_rate
count  288.000000
mean     3.330208
std      1.653322
min     -0.200000
25%      2.000000
50%      3.350000
75%      4.300000
max      7.900000

Column has no zero values?
date         True
wage_rate    True
dtype: bool


**The average growth in earnings year-over-year is 3.33 percent.**

In [40]:
# Save to csv
wage_monthly.to_csv('wage_rate_monthly_clean.csv', index=False)

### Wage Growth (Quarterly)

(Average Weekly Earnings: Whole Economy Year on Year Three Month Average Growth (%): Seasonally Adjusted Regular Pay Excluding Arrears)

Source: https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/timeseries/kai9/lms

In [41]:
# Load .csv
wage_quarterly = pd.read_csv('wage_rate.csv', skiprows=7)
wage_quarterly

,Important notes,Unnamed: 1
0,2001 MAR,3.9
1,2001 APR,4.5
2,2001 MAY,4.9
3,2001 JUN,5.0
4,2001 JUL,5.1
...,...,...
283,2024 OCT,5.4
284,2024 NOV,5.6
285,2024 DEC,5.9
286,2025 JAN,5.8


In [42]:
# Rename columns
wage_quarterly.rename(columns={'Important notes': 'date', 'Unnamed: 1': 'wage_rate'}, inplace=True)

In [43]:
# Convert 'date' column to datetime
wage_quarterly['date'] = pd.to_datetime(wage_quarterly['date'], format='%Y %b')
wage_quarterly['date'] = wage_quarterly['date'].dt.to_period('Q')
wage_quarterly

,date,wage_rate
0,2001Q1,3.9
1,2001Q2,4.5
2,2001Q2,4.9
3,2001Q2,5.0
4,2001Q3,5.1
...,...,...
283,2024Q4,5.4
284,2024Q4,5.6
285,2024Q4,5.9
286,2025Q1,5.8


In [44]:
# Group by quarter and calculate mean
wage_quarterly = wage_quarterly.groupby('date')['wage_rate'].mean().round(2).reset_index()
wage_quarterly

,date,wage_rate
0,2001Q1,3.90
1,2001Q2,4.80
2,2001Q3,5.20
3,2001Q4,4.83
4,2002Q1,4.70
...,...,...
92,2024Q1,6.03
93,2024Q2,5.73
94,2024Q3,5.17
95,2024Q4,5.63


In [45]:
validate_data(wage_quarterly)

Missing values per column:
date         0
wage_rate    0
dtype: int64

Unique values per column:
date         97
wage_rate    74
dtype: int64

Number of duplicate rows:
0

Summary statistics:
       wage_rate
count  97.000000
mean    3.342371
std     1.644454
min     0.600000
25%     2.030000
50%     3.370000
75%     4.270000
max     7.830000

Column has no zero values?
date         True
wage_rate    True
dtype: bool


In [46]:
# Save to csv
wage_quarterly.to_csv('wage_rate_quarterly.csv', index=False)

### FTSE 100

Source: https://tradingeconomics.com/united-kingdom/stock-market

In [47]:
# Load .csv
ftse100 = pd.read_csv('ftse100.csv')
ftse100

,Country,Category,DateTime,Close,Frequency,HistoricalDataSymbol,LastUpdate
0,United Kingdom,Stock Market,1986-03-04T00:00:00,1548.90000,Daily,UKX,2025-05-01T13:22:56
1,United Kingdom,Stock Market,1986-03-05T00:00:00,1569.10000,Daily,UKX,2025-05-01T13:22:56
2,United Kingdom,Stock Market,1986-03-06T00:00:00,1566.10000,Daily,UKX,2025-05-01T13:22:56
3,United Kingdom,Stock Market,1986-03-07T00:00:00,1573.80000,Daily,UKX,2025-05-01T13:22:56
4,United Kingdom,Stock Market,1986-03-10T00:00:00,1572.20000,Daily,UKX,2025-05-01T13:22:56
...,...,...,...,...,...,...,...
9995,United Kingdom,Stock Market,2025-04-25T00:00:00,8415.24994,Daily,UKX,2025-05-01T13:22:56
9996,United Kingdom,Stock Market,2025-04-28T00:00:00,8417.34046,Daily,UKX,2025-05-01T13:22:56
9997,United Kingdom,Stock Market,2025-04-29T00:00:00,8463.46245,Daily,UKX,2025-05-01T13:22:56
9998,United Kingdom,Stock Market,2025-04-30T00:00:00,8494.84565,Daily,UKX,2025-05-01T13:22:56


In [48]:
# Drop columns
ftse100 = ftse100.drop(columns=['Country', 'Category', 'Frequency', 'HistoricalDataSymbol', 'LastUpdate'])
ftse100

,DateTime,Close
0,1986-03-04T00:00:00,1548.90000
1,1986-03-05T00:00:00,1569.10000
2,1986-03-06T00:00:00,1566.10000
3,1986-03-07T00:00:00,1573.80000
4,1986-03-10T00:00:00,1572.20000
...,...,...
9995,2025-04-25T00:00:00,8415.24994
9996,2025-04-28T00:00:00,8417.34046
9997,2025-04-29T00:00:00,8463.46245
9998,2025-04-30T00:00:00,8494.84565


In [49]:
# Rename columns
ftse100.rename(columns={'DateTime': 'date', 'Close': 'ftse100'}, inplace=True)
ftse100

,date,ftse100
0,1986-03-04T00:00:00,1548.90000
1,1986-03-05T00:00:00,1569.10000
2,1986-03-06T00:00:00,1566.10000
3,1986-03-07T00:00:00,1573.80000
4,1986-03-10T00:00:00,1572.20000
...,...,...
9995,2025-04-25T00:00:00,8415.24994
9996,2025-04-28T00:00:00,8417.34046
9997,2025-04-29T00:00:00,8463.46245
9998,2025-04-30T00:00:00,8494.84565


In [50]:
# Change date to daily period data type
ftse100['date'] = pd.to_datetime(ftse100['date']).dt.to_period('D')
ftse100

,date,ftse100
0,1986-03-04,1548.90000
1,1986-03-05,1569.10000
2,1986-03-06,1566.10000
3,1986-03-07,1573.80000
4,1986-03-10,1572.20000
...,...,...
9995,2025-04-25,8415.24994
9996,2025-04-28,8417.34046
9997,2025-04-29,8463.46245
9998,2025-04-30,8494.84565


In [51]:
validate_data(ftse100)

Missing values per column:
date       0
ftse100    0
dtype: int64

Unique values per column:
date       10000
ftse100     9591
dtype: int64

Number of duplicate rows:
0

Summary statistics:
            ftse100
count  10000.000000
mean    5119.341922
std     1887.976603
min     1526.700000
25%     3522.625000
50%     5574.470000
75%     6588.415000
max     8871.307560

Column has no zero values?
date       True
ftse100    True
dtype: bool


In [52]:
# Save to csv
ftse100.to_csv('ftse_clean.csv', index=False)

### Inflation Expectations

(Citi/YouGov survey)

Source: https://tradingeconomics.com/united-kingdom/inflation-expectations

In [53]:
# Load .csv
infl_exp = pd.read_csv('inflation_expectations.csv')
infl_exp

,Country,Category,DateTime,Close,Frequency,HistoricalDataSymbol,LastUpdate
0,United Kingdom,Inflation Expectations,1999-12-31T00:00:00,1.5,Monthly,UNITEDKININFEXP,2019-03-26T17:55:00
1,United Kingdom,Inflation Expectations,2000-03-31T00:00:00,2.2,Monthly,UNITEDKININFEXP,2019-03-26T17:55:00
2,United Kingdom,Inflation Expectations,2000-06-30T00:00:00,2.4,Monthly,UNITEDKININFEXP,2019-03-26T17:55:00
3,United Kingdom,Inflation Expectations,2000-09-29T00:00:00,2.2,Monthly,UNITEDKININFEXP,2019-03-26T17:55:00
4,United Kingdom,Inflation Expectations,2000-12-29T00:00:00,2.3,Monthly,UNITEDKININFEXP,2019-03-26T17:55:00
...,...,...,...,...,...,...,...
249,United Kingdom,Inflation Expectations,2024-09-30T00:00:00,3.2,Monthly,UNITEDKININFEXP,2024-11-05T09:27:00
250,United Kingdom,Inflation Expectations,2024-10-31T00:00:00,3.3,Monthly,UNITEDKININFEXP,2024-11-05T09:27:00
251,United Kingdom,Inflation Expectations,2024-12-31T00:00:00,3.7,Monthly,UNITEDKININFEXP,2025-01-13T05:54:00
252,United Kingdom,Inflation Expectations,2025-01-31T00:00:00,3.5,Monthly,UNITEDKININFEXP,2025-02-06T10:47:00


In [54]:
# Drop columns
infl_exp = infl_exp.drop(columns=['Country', 'Category', 'Frequency',
                                  'HistoricalDataSymbol', 'LastUpdate'])
infl_exp

,DateTime,Close
0,1999-12-31T00:00:00,1.5
1,2000-03-31T00:00:00,2.2
2,2000-06-30T00:00:00,2.4
3,2000-09-29T00:00:00,2.2
4,2000-12-29T00:00:00,2.3
...,...,...
249,2024-09-30T00:00:00,3.2
250,2024-10-31T00:00:00,3.3
251,2024-12-31T00:00:00,3.7
252,2025-01-31T00:00:00,3.5


In [55]:
# Rename columns
infl_exp.rename(columns={'DateTime': 'date', 'Close': 'infl_exp'}, inplace=True)
infl_exp

,date,infl_exp
0,1999-12-31T00:00:00,1.5
1,2000-03-31T00:00:00,2.2
2,2000-06-30T00:00:00,2.4
3,2000-09-29T00:00:00,2.2
4,2000-12-29T00:00:00,2.3
...,...,...
249,2024-09-30T00:00:00,3.2
250,2024-10-31T00:00:00,3.3
251,2024-12-31T00:00:00,3.7
252,2025-01-31T00:00:00,3.5


In [56]:
# Change date to monthly period type
infl_exp['date'] = pd.to_datetime(infl_exp['date']).dt.to_period(freq="M")
infl_exp

,date,infl_exp
0,1999-12,1.5
1,2000-03,2.2
2,2000-06,2.4
3,2000-09,2.2
4,2000-12,2.3
...,...,...
249,2024-09,3.2
250,2024-10,3.3
251,2024-12,3.7
252,2025-01,3.5


In [57]:
validate_data(infl_exp)

Missing values per column:
date        0
infl_exp    0
dtype: int64

Unique values per column:
date        254
infl_exp     49
dtype: int64

Number of duplicate rows:
0

Summary statistics:
         infl_exp
count  254.000000
mean     2.824409
std      1.070090
min      0.800000
25%      2.300000
50%      2.600000
75%      3.200000
max      6.300000

Column has no zero values?
date        True
infl_exp    True
dtype: bool


In [58]:
# Save to csv
infl_exp.to_csv('infl_exp_clean.csv', index=False)

### PMI

S&P Global/CIPS United Kingdom Composite Purchasing Managers Index

Source: https://www.mql5.com/en/economic-calendar/united-kingdom/markit-composite-pmi

In [59]:
# Load .tsv
pmi = pd.read_table('pmi.tsv')
pmi

,Date,ActualValue,ForecastValue,PreviousValue
0,2025.04.23,48.2,NaN,51.5
1,2025.04.03,51.5,NaN,52.0
2,2025.03.24,52.0,NaN,50.5
3,2025.03.05,50.5,NaN,50.5
4,2025.02.21,50.5,NaN,50.6
...,...,...,...,...
130,2019.11.22,48.5,NaN,50.0
131,2019.11.05,50.0,NaN,49.3
132,2019.10.03,49.3,NaN,50.2
133,2019.09.04,50.2,NaN,50.7


In [60]:
# Drop columns
pmi = pmi.drop(columns=['ForecastValue', 'PreviousValue'])
pmi

,Date,ActualValue
0,2025.04.23,48.2
1,2025.04.03,51.5
2,2025.03.24,52.0
3,2025.03.05,50.5
4,2025.02.21,50.5
...,...,...
130,2019.11.22,48.5
131,2019.11.05,50.0
132,2019.10.03,49.3
133,2019.09.04,50.2


In [61]:
# Rename columns
pmi.rename(columns={'Date': 'date', 'ActualValue': 'pmi'}, inplace=True)
pmi

,date,pmi
0,2025.04.23,48.2
1,2025.04.03,51.5
2,2025.03.24,52.0
3,2025.03.05,50.5
4,2025.02.21,50.5
...,...,...
130,2019.11.22,48.5
131,2019.11.05,50.0
132,2019.10.03,49.3
133,2019.09.04,50.2


In [62]:
# Change date to datetime data type
pmi['date'] = pd.to_datetime(pmi['date'])
pmi

,date,pmi
0,2025-04-23,48.2
1,2025-04-03,51.5
2,2025-03-24,52.0
3,2025-03-05,50.5
4,2025-02-21,50.5
...,...,...
130,2019-11-22,48.5
131,2019-11-05,50.0
132,2019-10-03,49.3
133,2019-09-04,50.2


In [63]:
# Change date to monthly period
pmi['date'] = pmi['date'].dt.to_period('M')
pmi

,date,pmi
0,2025-04,48.2
1,2025-04,51.5
2,2025-03,52.0
3,2025-03,50.5
4,2025-02,50.5
...,...,...
130,2019-11,48.5
131,2019-11,50.0
132,2019-10,49.3
133,2019-09,50.2


In [64]:
# Group by month and calculate mean
pmi_monthly = pmi.groupby('date')['pmi'].mean().round(2).reset_index()
pmi_monthly

,date,pmi
0,2019-08,50.70
1,2019-09,50.20
2,2019-10,49.30
3,2019-11,49.25
4,2019-12,48.90
...,...,...
64,2024-12,50.50
65,2025-01,50.65
66,2025-02,50.55
67,2025-03,51.25


In [65]:
validate_data(pmi_monthly)

Missing values per column:
date    0
pmi     0
dtype: int64

Unique values per column:
date    69
pmi     62
dtype: int64

Number of duplicate rows:
0

Summary statistics:
             pmi
count  69.000000
mean   51.463768
std     6.408983
min    21.350000
25%    49.400000
50%    52.300000
75%    53.500000
max    62.300000

Column has no zero values?
date    True
pmi     True
dtype: bool


In [66]:
# Save to csv
pmi_monthly.to_csv('pmi_monthly_clean.csv', index=False)

### SONIA

(Sterling Overnight Index Average)

Source: https://www.bankofengland.co.uk/markets/sonia-benchmark

In [67]:
# Import .csv
sonia = pd.read_csv('sonia.csv')
sonia

,Date,Daily Sterling overnight index average (SONIA) rate [a] [b] IUDSOIA
0,01 May 25,4.4586
1,30 Apr 25,4.4592
2,29 Apr 25,4.4592
3,28 Apr 25,4.4590
4,25 Apr 25,4.4591
...,...,...
7153,08 Jan 97,5.9600
7154,07 Jan 97,5.9400
7155,06 Jan 97,5.9000
7156,03 Jan 97,6.0300


In [68]:
# Rename column headers
sonia.rename(columns={'Date': 'date',
                      'Daily Sterling overnight index average (SONIA) rate              [a] [b]             IUDSOIA': 'sonia'},
             inplace=True)
sonia

,date,sonia
0,01 May 25,4.4586
1,30 Apr 25,4.4592
2,29 Apr 25,4.4592
3,28 Apr 25,4.4590
4,25 Apr 25,4.4591
...,...,...
7153,08 Jan 97,5.9600
7154,07 Jan 97,5.9400
7155,06 Jan 97,5.9000
7156,03 Jan 97,6.0300


In [69]:
# Change date to daily period data type
sonia['date'] = pd.to_datetime(sonia['date']).dt.to_period('D')
sonia

/var/folders/hk/dv8kkk0x06n17jn62mhyd1cw0000gn/T/ipykernel_37434/1986764271.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sonia['date'] = pd.to_datetime(sonia['date']).dt.to_period('D')


,date,sonia
0,2025-05-01,4.4586
1,2025-04-30,4.4592
2,2025-04-29,4.4592
3,2025-04-28,4.4590
4,2025-04-25,4.4591
...,...,...
7153,1997-01-08,5.9600
7154,1997-01-07,5.9400
7155,1997-01-06,5.9000
7156,1997-01-03,6.0300


In [70]:
validate_data(sonia)

Missing values per column:
date     0
sonia    0
dtype: int64

Unique values per column:
date     7158
sonia    4606
dtype: int64

Number of duplicate rows:
0

Summary statistics:
             sonia
count  7158.000000
mean      2.799563
std       2.424763
min       0.039400
25%       0.455400
50%       3.028550
75%       4.980600
max       8.606800

Column has no zero values?
date     True
sonia    True
dtype: bool


In [71]:
# Save to csv
sonia.to_csv('sonia_clean.csv', index=False)

### FTSE VIX

Source: https://uk.investing.com/indices/ftse-100-vix-historical-data

In [72]:
# Import .csv
ftse_vix_monthly = pd.read_csv('ftse_vix_month.csv')
ftse_vix_monthly

,Date,Price,Open,High,Low,Vol.,Change %
0,01/06/2019,11.03,13.85,15.46,9.26,NaN,-18.58%
1,01/05/2019,13.55,11.90,16.82,6.58,NaN,16.41%
2,01/04/2019,11.64,12.64,13.54,8.04,NaN,-9.42%
3,01/03/2019,12.85,13.41,16.50,10.02,NaN,-4.74%
4,01/02/2019,13.49,12.55,17.31,3.45,NaN,-3.16%
...,...,...,...,...,...,...,...
174,01/12/2004,10.84,12.37,12.37,9.84,NaN,-14.24%
175,01/11/2004,12.64,14.36,14.36,11.18,NaN,-9.84%
176,01/10/2004,14.02,11.62,15.13,11.62,NaN,12.61%
177,01/09/2004,12.45,12.87,13.79,11.80,NaN,-6.67%


In [73]:
# Drop columns
ftse_vix_monthly = ftse_vix_monthly.drop(columns=['Open', 'High', 'Low', 'Vol.', 'Change %'])
ftse_vix_monthly

,Date,Price
0,01/06/2019,11.03
1,01/05/2019,13.55
2,01/04/2019,11.64
3,01/03/2019,12.85
4,01/02/2019,13.49
...,...,...
174,01/12/2004,10.84
175,01/11/2004,12.64
176,01/10/2004,14.02
177,01/09/2004,12.45


In [74]:
# Rename column headers
ftse_vix_monthly.rename(columns={'Date': 'date', 'Price': 'ftse_vix'}, inplace=True)
ftse_vix_monthly

,date,ftse_vix
0,01/06/2019,11.03
1,01/05/2019,13.55
2,01/04/2019,11.64
3,01/03/2019,12.85
4,01/02/2019,13.49
...,...,...
174,01/12/2004,10.84
175,01/11/2004,12.64
176,01/10/2004,14.02
177,01/09/2004,12.45


In [75]:
ftse_vix_monthly.dtypes

date         object
ftse_vix    float64
dtype: object

In [76]:
# Change date to datetime datatype
ftse_vix_monthly['date'] = pd.to_datetime(ftse_vix_monthly['date'], dayfirst=True)
ftse_vix_monthly

,date,ftse_vix
0,2019-06-01,11.03
1,2019-05-01,13.55
2,2019-04-01,11.64
3,2019-03-01,12.85
4,2019-02-01,13.49
...,...,...
174,2004-12-01,10.84
175,2004-11-01,12.64
176,2004-10-01,14.02
177,2004-09-01,12.45


In [77]:
# Change date to monthly period type
ftse_vix_monthly['date'] = ftse_vix_monthly['date'].dt.to_period(freq="M")
ftse_vix_monthly

,date,ftse_vix
0,2019-06,11.03
1,2019-05,13.55
2,2019-04,11.64
3,2019-03,12.85
4,2019-02,13.49
...,...,...
174,2004-12,10.84
175,2004-11,12.64
176,2004-10,14.02
177,2004-09,12.45


In [78]:
validate_data(ftse_vix_monthly)

Missing values per column:
date        0
ftse_vix    0
dtype: int64

Unique values per column:
date        179
ftse_vix    168
dtype: int64

Number of duplicate rows:
0

Summary statistics:
         ftse_vix
count  179.000000
mean    17.709106
std      7.169350
min      9.550000
25%     12.790000
50%     15.950000
75%     20.895000
max     54.150000

Column has no zero values?
date        True
ftse_vix    True
dtype: bool


In [79]:
# Save to csv
ftse_vix_monthly.to_csv('ftse_vix_monthly_clean.csv', index=False)

### VIX (Monthly)

Source: https://uk.investing.com/indices/volatility-s-p-500-historical-data

In [80]:
# Import .csv
vix_monthly = pd.read_csv('vix_monthly.csv')
vix_monthly

,Date,Price,Open,High,Low,Vol.,Change %
0,01/05/2025,18.14,23.94,25.62,17.15,NaN,-26.56%
1,01/04/2025,24.70,22.06,60.13,20.68,NaN,10.86%
2,01/03/2025,22.28,19.83,29.57,16.97,NaN,13.50%
3,01/02/2025,19.63,20.36,22.40,14.74,NaN,19.48%
4,01/01/2025,16.43,17.21,22.51,14.58,NaN,-5.30%
...,...,...,...,...,...,...,...
120,01/05/2015,13.84,13.98,16.36,11.82,NaN,-4.88%
121,01/04/2015,14.55,15.32,16.66,12.12,NaN,-4.84%
122,01/03/2015,15.29,13.90,17.19,12.54,NaN,14.62%
123,01/02/2015,13.34,20.89,22.81,12.86,NaN,-36.39%


In [81]:
# Drop columns
vix_monthly = vix_monthly.drop(columns=['Open', 'High', 'Low', 'Vol.', 'Change %'])
vix_monthly

,Date,Price
0,01/05/2025,18.14
1,01/04/2025,24.70
2,01/03/2025,22.28
3,01/02/2025,19.63
4,01/01/2025,16.43
...,...,...
120,01/05/2015,13.84
121,01/04/2015,14.55
122,01/03/2015,15.29
123,01/02/2015,13.34


In [82]:
# Rename column headers
vix_monthly.rename(columns={'Date': 'date', 'Price': 'vix_monthly'}, inplace=True)
vix_monthly

,date,vix_monthly
0,01/05/2025,18.14
1,01/04/2025,24.70
2,01/03/2025,22.28
3,01/02/2025,19.63
4,01/01/2025,16.43
...,...,...
120,01/05/2015,13.84
121,01/04/2015,14.55
122,01/03/2015,15.29
123,01/02/2015,13.34


In [83]:
# Change date to datetime datatype
vix_monthly['date'] = pd.to_datetime(vix_monthly['date'], dayfirst=True)
vix_monthly

,date,vix_monthly
0,2025-05-01,18.14
1,2025-04-01,24.70
2,2025-03-01,22.28
3,2025-02-01,19.63
4,2025-01-01,16.43
...,...,...
120,2015-05-01,13.84
121,2015-04-01,14.55
122,2015-03-01,15.29
123,2015-02-01,13.34


In [84]:
# Change date to monthly period type
vix_monthly['date'] = vix_monthly['date'].dt.to_period(freq="M")
vix_monthly

,date,vix_monthly
0,2025-05,18.14
1,2025-04,24.70
2,2025-03,22.28
3,2025-02,19.63
4,2025-01,16.43
...,...,...
120,2015-05,13.84
121,2015-04,14.55
122,2015-03,15.29
123,2015-02,13.34


In [85]:
validate_data(vix_monthly)

Missing values per column:
date           0
vix_monthly    0
dtype: int64

Unique values per column:
date           125
vix_monthly    120
dtype: int64

Number of duplicate rows:
0

Summary statistics:
       vix_monthly
count   125.000000
mean     18.529360
std       6.952124
min       9.510000
25%      13.540000
50%      16.480000
75%      20.970000
max      53.540000

Column has no zero values?
date           True
vix_monthly    True
dtype: bool


**From 2015 until the present the VIX had a monthly average high of 53.54 and a low of 9.51.**

In [86]:
# Save to csv
vix_monthly.to_csv('vix_monthly_clean.csv', index=False)

### VIX (Daily)

(Cboe Volatility Index: a gauge of U.S. equity market volatility)

Source: https://www.cboe.com/tradable_products/vix/vix_historical_data/

In [87]:
# Import .csv
vix = pd.read_csv('vix.csv')
vix

,DATE,OPEN,HIGH,LOW,CLOSE
0,01/02/1990,17.24,17.24,17.24,17.24
1,01/03/1990,18.19,18.19,18.19,18.19
2,01/04/1990,19.22,19.22,19.22,19.22
3,01/05/1990,20.11,20.11,20.11,20.11
4,01/08/1990,20.26,20.26,20.26,20.26
...,...,...,...,...,...
8917,05/01/2025,23.94,25.18,23.30,24.60
8918,05/02/2025,23.63,24.32,22.34,22.68
8919,05/05/2025,24.25,24.63,22.81,23.64
8920,05/06/2025,23.97,25.11,23.90,24.76


In [88]:
# Drop columns
vix = vix.drop(columns=['OPEN', 'HIGH', 'LOW'])
vix

,DATE,CLOSE
0,01/02/1990,17.24
1,01/03/1990,18.19
2,01/04/1990,19.22
3,01/05/1990,20.11
4,01/08/1990,20.26
...,...,...
8917,05/01/2025,24.60
8918,05/02/2025,22.68
8919,05/05/2025,23.64
8920,05/06/2025,24.76


In [89]:
# Rename column headers
vix.rename(columns={'DATE': 'date', 'CLOSE': 'vix'}, inplace=True)
vix

,date,vix
0,01/02/1990,17.24
1,01/03/1990,18.19
2,01/04/1990,19.22
3,01/05/1990,20.11
4,01/08/1990,20.26
...,...,...
8917,05/01/2025,24.60
8918,05/02/2025,22.68
8919,05/05/2025,23.64
8920,05/06/2025,24.76


In [90]:
# Change date to datetime datatype
vix['date'] = pd.to_datetime(vix['date'])
vix

,date,vix
0,1990-01-02,17.24
1,1990-01-03,18.19
2,1990-01-04,19.22
3,1990-01-05,20.11
4,1990-01-08,20.26
...,...,...
8917,2025-05-01,24.60
8918,2025-05-02,22.68
8919,2025-05-05,23.64
8920,2025-05-06,24.76


In [91]:
# Change date to monthly period type
vix['date'] = vix['date'].dt.to_period(freq="D")
vix

,date,vix
0,1990-01-02,17.24
1,1990-01-03,18.19
2,1990-01-04,19.22
3,1990-01-05,20.11
4,1990-01-08,20.26
...,...,...
8917,2025-05-01,24.60
8918,2025-05-02,22.68
8919,2025-05-05,23.64
8920,2025-05-06,24.76


In [92]:
validate_data(vix)

Missing values per column:
date    0
vix     0
dtype: int64

Unique values per column:
date    8922
vix     2521
dtype: int64

Number of duplicate rows:
0

Summary statistics:
               vix
count  8922.000000
mean     19.489124
std       7.840995
min       9.140000
25%      13.852500
50%      17.630000
75%      22.840000
max      82.690000

Column has no zero values?
date    True
vix     True
dtype: bool


**From 1990 until the present the VIX had a highest daily close of 82.69 and a lowest daily close of of 9.14.**

In [93]:
# Save to csv
vix.to_csv('vix_clean.csv', index=False)

### Monthly GDP/USD Exchange Rate

Source: [Bank of England](https://beta.bankofengland.co.uk/boeapps/database/fromshowcolumns.asp?Travel=NIxAZxSUx&FromSeries=1&ToSeries=50&DAT=ALL&FNY=&CSVF=TT&html.x=66&html.y=26&C=5YE&Filter=N)

In [94]:
# Import .csv
gbp_usd_monthly = pd.read_csv('gbp_usd.csv')
gbp_usd_monthly

,Date,"Monthly average Spot exchange rate, US$ into Sterling [a] XUMAUSS"
0,30 Apr 25,1.3131
1,31 Mar 25,1.2911
2,28 Feb 25,1.2545
3,31 Jan 25,1.2348
4,31 Dec 24,1.2647
...,...,...
599,31 May 75,2.3214
600,30 Apr 75,2.3708
601,31 Mar 75,2.4194
602,28 Feb 75,2.3950


In [95]:
# Rename column headers
gbp_usd_monthly.rename(columns={'Date': 'date',
                        'Monthly average Spot exchange rate, US$ into Sterling              [a]             XUMAUSS': 'gbp_usd_m'}, inplace=True)
gbp_usd_monthly

,date,gbp_usd_m
0,30 Apr 25,1.3131
1,31 Mar 25,1.2911
2,28 Feb 25,1.2545
3,31 Jan 25,1.2348
4,31 Dec 24,1.2647
...,...,...
599,31 May 75,2.3214
600,30 Apr 75,2.3708
601,31 Mar 75,2.4194
602,28 Feb 75,2.3950


In [96]:
# Change date to datetime datatype
gbp_usd_monthly['date'] = pd.to_datetime(gbp_usd_monthly['date'])
gbp_usd_monthly

/var/folders/hk/dv8kkk0x06n17jn62mhyd1cw0000gn/T/ipykernel_37434/2897397946.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  gbp_usd_monthly['date'] = pd.to_datetime(gbp_usd_monthly['date'])


,date,gbp_usd_m
0,2025-04-30,1.3131
1,2025-03-31,1.2911
2,2025-02-28,1.2545
3,2025-01-31,1.2348
4,2024-12-31,1.2647
...,...,...
599,1975-05-31,2.3214
600,1975-04-30,2.3708
601,1975-03-31,2.4194
602,1975-02-28,2.3950


In [97]:
# Change date to monthly period type
gbp_usd_monthly['date'] = gbp_usd_monthly['date'].dt.to_period(freq="M")
gbp_usd_monthly

,date,gbp_usd_m
0,2025-04,1.3131
1,2025-03,1.2911
2,2025-02,1.2545
3,2025-01,1.2348
4,2024-12,1.2647
...,...,...
599,1975-05,2.3214
600,1975-04,2.3708
601,1975-03,2.4194
602,1975-02,2.3950


In [98]:
validate_data(gbp_usd_monthly)

Missing values per column:
date         0
gbp_usd_m    0
dtype: int64

Unique values per column:
date         604
gbp_usd_m    576
dtype: int64

Number of duplicate rows:
0

Summary statistics:
        gbp_usd_m
count  604.000000
mean     1.620956
std      0.262844
min      1.093200
25%      1.440275
50%      1.595400
75%      1.771125
max      2.419400

Column has no zero values?
date         True
gbp_usd_m    True
dtype: bool


**The average monthly GBP/USD exchange rate from 1975 to the present is 1.62.**

In [99]:
# Save to csv
gbp_usd_monthly.to_csv('gbp_usd_monthly_clean.csv', index=False)

### Daily GDP/USD Exchange Rate

Source: https://fred.stlouisfed.org/series/DEXUSUK

In [100]:
# Import .csv
gbp_usd_daily = pd.read_csv('gbp_usd_daily.csv')
gbp_usd_daily

,observation_date,DEXUSUK
0,1997-01-02,1.6870
1,1997-01-03,1.6900
2,1997-01-06,1.6850
3,1997-01-07,1.6950
4,1997-01-08,1.6871
...,...,...
7392,2025-05-05,1.3284
7393,2025-05-06,1.3369
7394,2025-05-07,1.3344
7395,2025-05-08,1.3287


In [101]:
# Rename column headers
gbp_usd_daily.rename(columns={'observation_date': 'date',
                        'DEXUSUK': 'gbp_usd_d'}, inplace=True)
gbp_usd_daily

,date,gbp_usd_d
0,1997-01-02,1.6870
1,1997-01-03,1.6900
2,1997-01-06,1.6850
3,1997-01-07,1.6950
4,1997-01-08,1.6871
...,...,...
7392,2025-05-05,1.3284
7393,2025-05-06,1.3369
7394,2025-05-07,1.3344
7395,2025-05-08,1.3287


In [102]:
# Change date to datetime datatype
gbp_usd_daily['date'] = pd.to_datetime(gbp_usd_daily['date'])
gbp_usd_daily

,date,gbp_usd_d
0,1997-01-02,1.6870
1,1997-01-03,1.6900
2,1997-01-06,1.6850
3,1997-01-07,1.6950
4,1997-01-08,1.6871
...,...,...
7392,2025-05-05,1.3284
7393,2025-05-06,1.3369
7394,2025-05-07,1.3344
7395,2025-05-08,1.3287


In [103]:
# Change date to monthly period type
gbp_usd_daily['date'] = gbp_usd_daily['date'].dt.to_period(freq="D")
gbp_usd_daily

,date,gbp_usd_d
0,1997-01-02,1.6870
1,1997-01-03,1.6900
2,1997-01-06,1.6850
3,1997-01-07,1.6950
4,1997-01-08,1.6871
...,...,...
7392,2025-05-05,1.3284
7393,2025-05-06,1.3369
7394,2025-05-07,1.3344
7395,2025-05-08,1.3287


In [104]:
validate_data(gbp_usd_daily)

Missing values per column:
date           0
gbp_usd_d    284
dtype: int64

Unique values per column:
date         7397
gbp_usd_d    4318
dtype: int64

Number of duplicate rows:
0

Summary statistics:
         gbp_usd_d
count  7113.000000
mean      1.535900
std       0.214878
min       1.070300
25%       1.335600
50%       1.554200
75%       1.648600
max       2.110400

Column has no zero values?
date         True
gbp_usd_d    True
dtype: bool


**The average daily GBP/USD exchange rate from 1975 to the present is 1.54.**

In [105]:
# Save to csv
gbp_usd_daily.to_csv('gbp_usd_daily_clean.csv', index=False)

### Retail Sales Index

(All business all retail including fuel, seasonally adjusted)

rsi_mom: month on previous month % change

rsi_3mo3m: rolling 3 month on previous 3 month % change

rsi_3mo3m1y: rolling 3 month on same 3 month a year ago %

rsi_vol: volume 

Source: https://www.ons.gov.uk/businessindustryandtrade/retailindustry/datasets/retailsales

In [106]:
# Import .csv
retail = pd.read_csv('retail.csv')
retail

,Title,RSI:Month on prev month % change:All Business All retail inc fuel VOL SA,RSI:Rolling 3mnth on prev 3mnth %change:All Business All retail inc fuel VOL SA,RSI:All retail inc fuel:All Business:VOL SA:Rolling 3m on same 3m a year ago %,RSI:Volume Seasonally Adjusted:All Retailers inc fuel:All Business Index
0,CDID,J5EC,J5EG,J5EH,J5EK
1,PreUnit,NaN,NaN,NaN,NaN
2,Unit,%,%,%,"Index, base year = 100"
3,Release Date,25-04-2025,25-04-2025,25-04-2025,25-04-2025
4,Next release,23-May-25,23-May-25,23-May-25,23-May-25
...,...,...,...,...,...
634,2024 NOV,-0.1,0.3,1.3,97.7
635,2024 DEC,-0.4,-0.7,1.5,97.3
636,2025 JAN,1.3,-0.5,1.3,98.6
637,2025 FEB,0.7,0.3,1.9,99.3


In [107]:
# Rename column headers
retail.rename(columns={'Title': 'date',
                        'RSI:Month on prev month % change:All Business All retail inc fuel VOL SA': 'rsi_mom',
                        'RSI:Rolling 3mnth on prev 3mnth %change:All Business All retail inc fuel VOL SA': 'rsi_3mo3m',
                        'RSI:All retail inc fuel:All Business:VOL SA:Rolling 3m on same 3m a year ago %': 'rsi_3mo3m1y',
                        'RSI:Volume Seasonally Adjusted:All Retailers inc fuel:All Business Index': 'rsi_vol'}, inplace=True)
retail

,date,rsi_mom,rsi_3mo3m,rsi_3mo3m1y,rsi_vol
0,CDID,J5EC,J5EG,J5EH,J5EK
1,PreUnit,NaN,NaN,NaN,NaN
2,Unit,%,%,%,"Index, base year = 100"
3,Release Date,25-04-2025,25-04-2025,25-04-2025,25-04-2025
4,Next release,23-May-25,23-May-25,23-May-25,23-May-25
...,...,...,...,...,...
634,2024 NOV,-0.1,0.3,1.3,97.7
635,2024 DEC,-0.4,-0.7,1.5,97.3
636,2025 JAN,1.3,-0.5,1.3,98.6
637,2025 FEB,0.7,0.3,1.9,99.3


In [108]:
# Split dataframe by annual, quarterly, and monthly values
rsi_annual, rsi_quarterly, rsi_monthly = split_dataframe(retail, 'date', prefix='rsi')

Original dataset rows: 639
Annual dataset rows: 37
Quarterly dataset rows: 149
Monthly dataset rows: 447
Sample of unmatched dates:
0            CDID
1         PreUnit
2            Unit
3    Release Date
4    Next release
Name: date, dtype: object


In [109]:
# Check for missing values
rsi_quarterly.isnull().sum()

date             0
rsi_mom        149
rsi_3mo3m       33
rsi_3mo3m1y    149
rsi_vol         32
dtype: int64

There are no values in rsi_mom or rsi_3mo3m1y so these columns can be dropped.

In [110]:
# drop columns with no values
rsi_quarterly = rsi_quarterly.drop(['rsi_mom', 'rsi_3mo3m1y'], axis=1)
rsi_quarterly

,date,rsi_3mo3m,rsi_vol
43,1988Q1,NaN,NaN
44,1988Q2,NaN,NaN
45,1988Q3,NaN,NaN
46,1988Q4,NaN,NaN
47,1989Q1,NaN,NaN
...,...,...,...
187,2024Q1,1.5,97.6
188,2024Q2,-0.5,97.1
189,2024Q3,1.3,98.3
190,2024Q4,-0.7,97.6


In [111]:
# Check for missing values
rsi_monthly.isnull().sum()

date             0
rsi_mom         97
rsi_3mo3m      101
rsi_3mo3m1y    110
rsi_vol         96
dtype: int64

In [112]:
rsi_quarterly[['rsi_3mo3m', 'rsi_vol']] = rsi_quarterly[['rsi_3mo3m', 'rsi_vol']].astype(float)
rsi_quarterly.dtypes

date         period[Q-DEC]
rsi_3mo3m          float64
rsi_vol            float64
dtype: object

In [113]:
rsi_monthly[['rsi_mom', 'rsi_3mo3m', 'rsi_3mo3m1y', 'rsi_vol']] = rsi_monthly[['rsi_mom', 'rsi_3mo3m', 'rsi_3mo3m1y', 'rsi_vol']].astype(float)
rsi_monthly.dtypes

date           period[M]
rsi_mom          float64
rsi_3mo3m        float64
rsi_3mo3m1y      float64
rsi_vol          float64
dtype: object

In [114]:
validate_data(rsi_quarterly)

Missing values per column:
date          0
rsi_3mo3m    33
rsi_vol      32
dtype: int64

Unique values per column:
date         149
rsi_3mo3m     44
rsi_vol       99
dtype: int64

Number of duplicate rows:
0

Summary statistics:
        rsi_3mo3m     rsi_vol
count  116.000000  117.000000
mean     0.498276   83.852991
std      2.474952   12.953848
min     -9.400000   57.700000
25%     -0.400000   77.100000
50%      0.500000   83.500000
75%      1.300000   95.800000
max     18.300000  108.000000

Column has no zero values?
date          True
rsi_3mo3m    False
rsi_vol       True
dtype: bool


In [115]:
validate_data(rsi_monthly)

Missing values per column:
date             0
rsi_mom         97
rsi_3mo3m      101
rsi_3mo3m1y    110
rsi_vol         96
dtype: int64

Unique values per column:
date           447
rsi_mom         62
rsi_3mo3m       68
rsi_3mo3m1y    111
rsi_vol        216
dtype: int64

Number of duplicate rows:
0

Summary statistics:
          rsi_mom   rsi_3mo3m  rsi_3mo3m1y     rsi_vol
count  350.000000  346.000000    337.00000  351.000000
mean     0.179143    0.480925      1.88635   83.839031
std      1.890026    2.285654      3.60956   12.964055
min    -17.800000  -13.400000    -13.10000   56.600000
25%     -0.400000   -0.200000      0.00000   76.850000
50%      0.200000    0.500000      2.00000   83.600000
75%      0.700000    1.200000      3.80000   95.900000
max     13.700000   18.300000     21.10000  109.200000

Column has no zero values?
date            True
rsi_mom        False
rsi_3mo3m      False
rsi_3mo3m1y    False
rsi_vol         True
dtype: bool


**These columns were taken from a spreadsheet with dozens of columns so it is not suprising that there are missing values but this could cause complications when using these features in predictive modeling.**

### Mortgage Interest Rates

2 year (75% loan-to-value) fixed rate mortgage, not seasonally adjusted

standard variable rate mortgage, not seasonally adjusted

Source: [Bank of England](https://www.bankofengland.co.uk/boeapps/database/fromshowcolumns.asp?Travel=NIxSTxTIxSUx&FromSeries=1&ToSeries=50&DAT=ALL&FNY=&CSVF=TT&html.x=117&html.y=47&C=EOT&C=RO&Filter=N)

In [116]:
# Import .csv
mortgage_rates = pd.read_csv('mortgage_rates.csv')
mortgage_rates

,Date,Monthly interest rate of UK monetary financial institutions (excl. Central Bank) sterling 2 year (75% LTV) fixed rate mortgage to households (in percent) not seasonally adjusted [a] [b] [c] [d] IUMBV34,Monthly interest rate of UK monetary financial institutions (excl. Central Bank) sterling revert-to-rate mortgage to households (in percent) not seasonally adjusted [a] [c] [d] IUMTLMV
0,30 Apr 25,4.43,7.21
1,31 Mar 25,4.54,7.23
2,28 Feb 25,4.66,7.33
3,31 Jan 25,4.64,7.46
4,31 Dec 24,4.60,7.47
...,...,...,...
359,31 May 95,8.22,8.36
360,30 Apr 95,8.29,8.36
361,31 Mar 95,8.08,8.36
362,28 Feb 95,8.38,8.33


In [117]:
# Rename column headers
mortgage_rates.rename(columns={'Date': 'date',
                        'Monthly interest rate of UK monetary financial institutions (excl. Central Bank) sterling 2 year (75% LTV) fixed rate mortgage to households (in percent) not seasonally adjusted              [a] [b] [c] [d]             IUMBV34': 'mortgage_2yr',
                        'Monthly interest rate of UK monetary financial institutions (excl. Central Bank) sterling revert-to-rate mortgage to households (in percent) not seasonally adjusted              [a] [c] [d]             IUMTLMV': 'mortgage_svr'},
                      inplace=True)
mortgage_rates

,date,mortgage_2yr,mortgage_svr
0,30 Apr 25,4.43,7.21
1,31 Mar 25,4.54,7.23
2,28 Feb 25,4.66,7.33
3,31 Jan 25,4.64,7.46
4,31 Dec 24,4.60,7.47
...,...,...,...
359,31 May 95,8.22,8.36
360,30 Apr 95,8.29,8.36
361,31 Mar 95,8.08,8.36
362,28 Feb 95,8.38,8.33


In [118]:
mortgage_rates.dtypes

date             object
mortgage_2yr    float64
mortgage_svr    float64
dtype: object

In [119]:
# Change date to datetime datatype
mortgage_rates['date'] = pd.to_datetime(mortgage_rates['date'])
mortgage_rates

/var/folders/hk/dv8kkk0x06n17jn62mhyd1cw0000gn/T/ipykernel_37434/63965167.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  mortgage_rates['date'] = pd.to_datetime(mortgage_rates['date'])


,date,mortgage_2yr,mortgage_svr
0,2025-04-30,4.43,7.21
1,2025-03-31,4.54,7.23
2,2025-02-28,4.66,7.33
3,2025-01-31,4.64,7.46
4,2024-12-31,4.60,7.47
...,...,...,...
359,1995-05-31,8.22,8.36
360,1995-04-30,8.29,8.36
361,1995-03-31,8.08,8.36
362,1995-02-28,8.38,8.33


In [120]:
# Change date to monthly period type
mortgage_rates['date'] = mortgage_rates['date'].dt.to_period(freq="M")
mortgage_rates

,date,mortgage_2yr,mortgage_svr
0,2025-04,4.43,7.21
1,2025-03,4.54,7.23
2,2025-02,4.66,7.33
3,2025-01,4.64,7.46
4,2024-12,4.60,7.47
...,...,...,...
359,1995-05,8.22,8.36
360,1995-04,8.29,8.36
361,1995-03,8.08,8.36
362,1995-02,8.38,8.33


In [121]:
validate_data(mortgage_rates)

Missing values per column:
date            0
mortgage_2yr    0
mortgage_svr    0
dtype: int64

Unique values per column:
date            364
mortgage_2yr    254
mortgage_svr    184
dtype: int64

Number of duplicate rows:
0

Summary statistics:
       mortgage_2yr  mortgage_svr
count    364.000000    364.000000
mean       4.271923      5.759533
std        1.961094      1.591860
min        1.200000      3.590000
25%        2.365000      4.290000
50%        4.550000      5.645000
75%        5.780000      7.220000
max        8.380000      8.870000

Column has no zero values?
date            True
mortgage_2yr    True
mortgage_svr    True
dtype: bool


**The average 2-year mortgage rate is 4.27% and the average variable mortage rate is 5.76% from 1995 to the present. Note that including both of these features in a model might lead to multicollinearity issues.**

In [122]:
# Save to csv
mortgage_rates.to_csv('mortgage_rates_clean.csv', index=False)

### Household Debt

(Debt to Income Ratio)

Source: https://www.ons.gov.uk/economy/grossdomesticproductgdp/timeseries/cvzi/ukea

In [123]:
# Import .csv
household_debt = pd.read_csv('household_debt.csv', skiprows=7)
household_debt

,Important notes,Unnamed: 1
0,1987 Q1,76.3
1,1987 Q2,78.2
2,1987 Q3,80.2
3,1987 Q4,82.5
4,1988 Q1,83.4
...,...,...
147,2023 Q4,123.8
148,2024 Q1,122.3
149,2024 Q2,121.0
150,2024 Q3,120.2


In [124]:
# Rename columns
household_debt.rename(columns={'Important notes': 'date', 'Unnamed: 1': 'household_debt'}, inplace=True)
household_debt

,date,household_debt
0,1987 Q1,76.3
1,1987 Q2,78.2
2,1987 Q3,80.2
3,1987 Q4,82.5
4,1988 Q1,83.4
...,...,...
147,2023 Q4,123.8
148,2024 Q1,122.3
149,2024 Q2,121.0
150,2024 Q3,120.2


In [125]:
# Change date data type to quarterly period
household_debt['date'] = pd.PeriodIndex(household_debt['date'].str.replace(' ', '', regex=False), freq='Q')
household_debt

,date,household_debt
0,1987Q1,76.3
1,1987Q2,78.2
2,1987Q3,80.2
3,1987Q4,82.5
4,1988Q1,83.4
...,...,...
147,2023Q4,123.8
148,2024Q1,122.3
149,2024Q2,121.0
150,2024Q3,120.2


In [126]:
validate_data(household_debt)

Missing values per column:
date              0
household_debt    0
dtype: int64

Unique values per column:
date              152
household_debt    129
dtype: int64

Number of duplicate rows:
0

Summary statistics:
       household_debt
count      152.000000
mean       119.596711
std         24.919006
min         76.300000
25%         92.425000
50%        130.150000
75%        138.800000
max        156.400000

Column has no zero values?
date              True
household_debt    True
dtype: bool


**The average debt-to-income ratio from 1987 to the present.**

In [127]:
# Save to csv
household_debt.to_csv('household_debt_clean.csv', index=False)

**Creating a monthly version of the dataframe to be included with other monthly economic indicators:**

In [128]:
# Convert quarterly data to monthly
household_debt_monthly = period_quarterly_to_monthly(household_debt, 'date')
household_debt_monthly

,date,household_debt
0,1987-01,76.3
0,1987-02,76.3
0,1987-03,76.3
1,1987-04,78.2
1,1987-05,78.2
...,...,...
150,2024-08,120.2
150,2024-09,120.2
151,2024-10,118.1
151,2024-11,118.1


In [129]:
# Save to csv
household_debt_monthly.to_csv('household_debt_monthly_clean.csv', index=False)

### FTSE 250

In [130]:
# Import .csv
ftse250 = pd.read_csv('ftse_250_clean.csv')
ftse250

,date,price
0,1989-12-29,"2,661.76"
1,1990-01-02,"2,678.94"
2,1990-01-03,"2,723.30"
3,1990-01-04,"2,737.81"
4,1990-01-05,"2,725.35"
...,...,...
8923,2025-04-24,"19,504.37"
8924,2025-04-25,"19,609.69"
8925,2025-04-28,"19,733.31"
8926,2025-04-29,"19,809.72"


In [131]:
# Rename columns
ftse250.rename(columns={'price': 'ftse250'}, inplace=True)
ftse250

,date,ftse250
0,1989-12-29,"2,661.76"
1,1990-01-02,"2,678.94"
2,1990-01-03,"2,723.30"
3,1990-01-04,"2,737.81"
4,1990-01-05,"2,725.35"
...,...,...
8923,2025-04-24,"19,504.37"
8924,2025-04-25,"19,609.69"
8925,2025-04-28,"19,733.31"
8926,2025-04-29,"19,809.72"


In [132]:
# Change date to daily period data type
ftse250['date'] = pd.to_datetime(ftse250['date']).dt.to_period('D')
ftse250

,date,ftse250
0,1989-12-29,"2,661.76"
1,1990-01-02,"2,678.94"
2,1990-01-03,"2,723.30"
3,1990-01-04,"2,737.81"
4,1990-01-05,"2,725.35"
...,...,...
8923,2025-04-24,"19,504.37"
8924,2025-04-25,"19,609.69"
8925,2025-04-28,"19,733.31"
8926,2025-04-29,"19,809.72"


In [133]:
ftse250['ftse250'] = ftse250['ftse250'].str.replace(',', '').astype(float)
ftse250.dtypes

date       period[D]
ftse250      float64
dtype: object

In [134]:
validate_data(ftse250)

Missing values per column:
date       0
ftse250    0
dtype: int64

Unique values per column:
date       8928
ftse250    8886
dtype: int64

Number of duplicate rows:
0

Summary statistics:
            ftse250
count   8928.000000
mean   10595.070865
std     6485.312480
min     1967.570000
25%     4742.427500
50%     9236.215000
75%    17241.605000
max    24250.830000

Column has no zero values?
date       True
ftse250    True
dtype: bool


In [135]:
# Save to csv
ftse250.to_csv('ftse250_clean.csv', index=False)

### FTSE 250 Monthly

Source: https://uk.investing.com/indices/uk-250-historical-data

In [136]:
# Import .csv
ftse250_monthly = pd.read_csv('ftse250_monthly.csv')
ftse250_monthly

,Date,Price,Open,High,Low,Vol.,Change %
0,01/05/2025,"20,961.09","19,884.59","20,973.86","19,882.06",4.29B,5.41%
1,01/04/2025,"19,884.59","19,475.48","19,948.64","17,392.49",7.72B,2.10%
2,01/03/2025,"19,475.48","20,326.38","20,468.65","19,449.99",8.08B,-4.19%
3,01/02/2025,"20,326.38","20,950.48","21,117.98","20,236.76",6.84B,-2.98%
4,01/01/2025,"20,950.48","20,622.61","20,950.48","19,659.02",6.87B,1.59%
...,...,...,...,...,...,...,...
120,01/05/2015,"18,154.42","17,474.63","18,295.12","17,220.51",4.72B,3.89%
121,01/04/2015,"17,474.63","17,090.64","17,904.33","17,039.55",5.04B,2.25%
122,01/03/2015,"17,090.64","17,273.82","17,655.81","16,818.50",7.11B,-1.06%
123,01/02/2015,"17,273.82","16,305.77","17,277.92","16,262.15",6.96B,5.94%


In [137]:
# Drop columns
ftse250_monthly = ftse250_monthly.drop(columns=['Open', 'High', 'Low', 'Vol.', 'Change %'])
ftse250_monthly

,Date,Price
0,01/05/2025,"20,961.09"
1,01/04/2025,"19,884.59"
2,01/03/2025,"19,475.48"
3,01/02/2025,"20,326.38"
4,01/01/2025,"20,950.48"
...,...,...
120,01/05/2015,"18,154.42"
121,01/04/2015,"17,474.63"
122,01/03/2015,"17,090.64"
123,01/02/2015,"17,273.82"


In [138]:
# Rename column headers
ftse250_monthly.rename(columns={'Date': 'date', 'Price': 'ftse250_monthly'}, inplace=True)
ftse250_monthly

,date,ftse250_monthly
0,01/05/2025,"20,961.09"
1,01/04/2025,"19,884.59"
2,01/03/2025,"19,475.48"
3,01/02/2025,"20,326.38"
4,01/01/2025,"20,950.48"
...,...,...
120,01/05/2015,"18,154.42"
121,01/04/2015,"17,474.63"
122,01/03/2015,"17,090.64"
123,01/02/2015,"17,273.82"


In [139]:
# Change date to datetime datatype
ftse250_monthly['date'] = pd.to_datetime(ftse250_monthly['date'], dayfirst=True)
ftse250_monthly

,date,ftse250_monthly
0,2025-05-01,"20,961.09"
1,2025-04-01,"19,884.59"
2,2025-03-01,"19,475.48"
3,2025-02-01,"20,326.38"
4,2025-01-01,"20,950.48"
...,...,...
120,2015-05-01,"18,154.42"
121,2015-04-01,"17,474.63"
122,2015-03-01,"17,090.64"
123,2015-02-01,"17,273.82"


In [140]:
# Change date to monthly period type
ftse250_monthly['date'] = ftse250_monthly['date'].dt.to_period(freq="M")
ftse250_monthly

,date,ftse250_monthly
0,2025-05,"20,961.09"
1,2025-04,"19,884.59"
2,2025-03,"19,475.48"
3,2025-02,"20,326.38"
4,2025-01,"20,950.48"
...,...,...
120,2015-05,"18,154.42"
121,2015-04,"17,474.63"
122,2015-03,"17,090.64"
123,2015-02,"17,273.82"


In [141]:
# Remove commas and convert from string to float
ftse250_monthly['ftse250_monthly'] = ftse250_monthly['ftse250_monthly'].str.replace(',', '').astype(float)
ftse250_monthly.dtypes

date               period[M]
ftse250_monthly      float64
dtype: object

In [142]:
validate_data(ftse250_monthly)

Missing values per column:
date               0
ftse250_monthly    0
dtype: int64

Unique values per column:
date               125
ftse250_monthly    125
dtype: int64

Number of duplicate rows:
0

Summary statistics:
       ftse250_monthly
count       125.000000
mean      19340.083600
std        1779.353347
min       15101.130000
25%       17788.330000
50%       19425.140000
75%       20488.300000
max       24102.190000

Column has no zero values?
date               True
ftse250_monthly    True
dtype: bool


In [143]:
# Save to csv
ftse250_monthly.to_csv('ftse250_monthly_clean.csv', index=False)

### FTSE All Share Monthly

Source: https://uk.investing.com/indices/ftse-all-share-historical-data

In [144]:
# Import .csv
ftse_all_share_monthly = pd.read_csv('ftse_all_share_monthly.csv')
ftse_all_share_monthly

,Date,Price,Open,High,Low,Vol.,Change %
0,01/05/2025,"4,759.54","4,594.05","4,782.21","4,582.70",2.76B,3.60%
1,01/04/2025,"4,594.05","4,623.62","4,669.66","4,074.68",3.22B,-0.64%
2,01/03/2025,"4,623.62","4,754.32","4,803.89","4,584.09",3.19B,-2.75%
3,01/02/2025,"4,754.32","4,710.58","4,778.92","4,627.36",2.84B,0.93%
4,01/01/2025,"4,710.58","4,467.80","4,717.02","4,448.27",2.74B,5.43%
...,...,...,...,...,...,...,...
266,01/03/2003,"1,735.70","1,759.20","1,853.60","1,590.30",NaN,-1.33%
267,01/02/2003,"1,759.10","1,722.70","1,798.80","1,707.60",NaN,2.14%
268,01/01/2003,"1,722.30","1,893.80","1,932.30","1,650.60",NaN,-9.05%
269,01/12/2002,"1,893.70","2,003.10","2,025.30","1,834.10",NaN,-5.46%


In [145]:
# Drop columns
ftse_all_share_monthly = ftse_all_share_monthly.drop(columns=['Open', 'High', 'Low', 'Vol.', 'Change %'])
ftse_all_share_monthly

,Date,Price
0,01/05/2025,"4,759.54"
1,01/04/2025,"4,594.05"
2,01/03/2025,"4,623.62"
3,01/02/2025,"4,754.32"
4,01/01/2025,"4,710.58"
...,...,...
266,01/03/2003,"1,735.70"
267,01/02/2003,"1,759.10"
268,01/01/2003,"1,722.30"
269,01/12/2002,"1,893.70"


In [146]:
# Rename column headers
ftse_all_share_monthly.rename(columns={'Date': 'date', 'Price': 'ftse_all_share_monthly'}, inplace=True)
ftse_all_share_monthly

,date,ftse_all_share_monthly
0,01/05/2025,"4,759.54"
1,01/04/2025,"4,594.05"
2,01/03/2025,"4,623.62"
3,01/02/2025,"4,754.32"
4,01/01/2025,"4,710.58"
...,...,...
266,01/03/2003,"1,735.70"
267,01/02/2003,"1,759.10"
268,01/01/2003,"1,722.30"
269,01/12/2002,"1,893.70"


In [147]:
# Change date to datetime datatype
ftse_all_share_monthly['date'] = pd.to_datetime(ftse_all_share_monthly['date'], dayfirst=True)
ftse_all_share_monthly

,date,ftse_all_share_monthly
0,2025-05-01,"4,759.54"
1,2025-04-01,"4,594.05"
2,2025-03-01,"4,623.62"
3,2025-02-01,"4,754.32"
4,2025-01-01,"4,710.58"
...,...,...
266,2003-03-01,"1,735.70"
267,2003-02-01,"1,759.10"
268,2003-01-01,"1,722.30"
269,2002-12-01,"1,893.70"


In [148]:
# Change date to monthly period type
ftse_all_share_monthly['date'] = ftse_all_share_monthly['date'].dt.to_period(freq="M")
ftse_all_share_monthly

,date,ftse_all_share_monthly
0,2025-05,"4,759.54"
1,2025-04,"4,594.05"
2,2025-03,"4,623.62"
3,2025-02,"4,754.32"
4,2025-01,"4,710.58"
...,...,...
266,2003-03,"1,735.70"
267,2003-02,"1,759.10"
268,2003-01,"1,722.30"
269,2002-12,"1,893.70"


In [149]:
validate_data(ftse_all_share_monthly)

Missing values per column:
date                      0
ftse_all_share_monthly    0
dtype: int64

Unique values per column:
date                      271
ftse_all_share_monthly    271
dtype: int64

Number of duplicate rows:
0

Summary statistics:
           date ftse_all_share_monthly
count       271                    271
unique      271                    271
top     2025-05               4,759.54
freq          1                      1

Column has no zero values?
date                      True
ftse_all_share_monthly    True
dtype: bool


In [150]:
# Save to csv
ftse_all_share_monthly.to_csv('ftse_all_share_monthly_clean.csv', index=False)

### FTSE 100 Historical Volatility

Source: https://www.tradingview.com/chart/rqC104t4/?symbol=FTSE%3AUKX

In [151]:
# Import .csv
ftse100_volat = pd.read_csv('ftse100_volat.csv')
ftse100_volat

,time,open,high,low,close,HV
0,1999-03-01,6175.1001,6365.3999,5968.2002,6295.2002,34.822327
1,1999-04-01,6295.2002,6635.7998,6267.5000,6552.2002,35.336627
2,1999-05-03,6552.2002,6663.7002,6110.0000,6226.0000,37.861787
3,1999-06-01,6226.0000,6614.0000,6226.0000,6319.0000,26.463953
4,1999-07-01,6318.5000,6649.1001,6092.1001,6231.7998,24.459283
...,...,...,...,...,...,...
310,2025-01-02,8173.0200,8692.8400,8160.6000,8673.9600,17.061888
311,2025-02-03,8673.9600,8820.9300,8520.2000,8809.7400,16.771649
312,2025-03-03,8809.7400,8908.8200,8481.1100,8582.8100,18.130499
313,2025-04-01,8582.8100,8671.9100,7544.8300,8494.8500,17.983591


In [152]:
# Drop columns
ftse100_volat = ftse100_volat.drop(columns=['open', 'high', 'low'])
ftse100_volat

,time,close,HV
0,1999-03-01,6295.2002,34.822327
1,1999-04-01,6552.2002,35.336627
2,1999-05-03,6226.0000,37.861787
3,1999-06-01,6319.0000,26.463953
4,1999-07-01,6231.7998,24.459283
...,...,...,...
310,2025-01-02,8673.9600,17.061888
311,2025-02-03,8809.7400,16.771649
312,2025-03-03,8582.8100,18.130499
313,2025-04-01,8494.8500,17.983591


In [153]:
# Rename column headers
ftse100_volat.rename(columns={'time': 'date', 'close': 'ftse100_monthly', 'HV': 'ftse100_volat'}, inplace=True)
ftse100_volat

,date,ftse100_monthly,ftse100_volat
0,1999-03-01,6295.2002,34.822327
1,1999-04-01,6552.2002,35.336627
2,1999-05-03,6226.0000,37.861787
3,1999-06-01,6319.0000,26.463953
4,1999-07-01,6231.7998,24.459283
...,...,...,...
310,2025-01-02,8673.9600,17.061888
311,2025-02-03,8809.7400,16.771649
312,2025-03-03,8582.8100,18.130499
313,2025-04-01,8494.8500,17.983591


In [154]:
# Change date to datetime datatype
ftse100_volat['date'] = pd.to_datetime(ftse100_volat['date'], format='%Y-%m-%d', dayfirst=True)
ftse100_volat

,date,ftse100_monthly,ftse100_volat
0,1999-03-01,6295.2002,34.822327
1,1999-04-01,6552.2002,35.336627
2,1999-05-03,6226.0000,37.861787
3,1999-06-01,6319.0000,26.463953
4,1999-07-01,6231.7998,24.459283
...,...,...,...
310,2025-01-02,8673.9600,17.061888
311,2025-02-03,8809.7400,16.771649
312,2025-03-03,8582.8100,18.130499
313,2025-04-01,8494.8500,17.983591


In [155]:
# Change date to monthly period type
ftse100_volat['date'] = ftse100_volat['date'].dt.to_period(freq="M")
ftse100_volat

,date,ftse100_monthly,ftse100_volat
0,1999-03,6295.2002,34.822327
1,1999-04,6552.2002,35.336627
2,1999-05,6226.0000,37.861787
3,1999-06,6319.0000,26.463953
4,1999-07,6231.7998,24.459283
...,...,...,...
310,2025-01,8673.9600,17.061888
311,2025-02,8809.7400,16.771649
312,2025-03,8582.8100,18.130499
313,2025-04,8494.8500,17.983591


In [156]:
validate_data(ftse100_volat)

Missing values per column:
date               0
ftse100_monthly    0
ftse100_volat      0
dtype: int64

Unique values per column:
date               315
ftse100_monthly    315
ftse100_volat      315
dtype: int64

Number of duplicate rows:
0

Summary statistics:
       ftse100_monthly  ftse100_volat
count       315.000000     315.000000
mean       6202.116631      24.921943
std        1127.286191       9.564923
min        3567.399900       9.240783
25%        5516.844950      17.132383
50%        6255.700200      23.504062
75%        7058.030050      30.355544
max        8809.740000      51.757971

Column has no zero values?
date               True
ftse100_monthly    True
ftse100_volat      True
dtype: bool


In [157]:
# Save to csv
ftse100_volat.to_csv('ftse100_volat_clean.csv', index=False)

### FTSE 250 Historical Volatility

Source: https://www.tradingview.com/chart/rqC104t4/?symbol=FTSE%3AMCX

In [158]:
# Import .csv
ftse250_volat = pd.read_csv('ftse250_volat.csv')
ftse250_volat

,time,open,high,low,close,HV
0,1998-10-01,4524.7002,4811.3999,4247.6001,4811.3999,NaN
1,1998-11-02,4815.2002,4975.2998,4772.2002,4907.2998,NaN
2,1998-12-01,4883.1001,4883.1001,4668.6001,4854.7002,NaN
3,1999-01-01,4856.7002,5026.8999,4825.6001,5024.2002,NaN
4,1999-02-01,4901.2998,5252.0000,4901.2998,5250.5000,NaN
...,...,...,...,...,...,...
314,2025-01-02,20622.6100,20950.4800,19659.0200,20950.4800,20.200117
315,2025-02-03,20950.4800,21117.9800,20236.7600,20326.3800,21.619851
316,2025-03-03,20326.3800,20468.6500,19449.9900,19475.4800,21.689952
317,2025-04-01,19475.4800,19948.6400,17392.4900,19884.5900,22.063025


In [159]:
# Drop columns
ftse250_volat = ftse250_volat.drop(columns=['open', 'high', 'low', 'close'])
ftse250_volat

,time,HV
0,1998-10-01,NaN
1,1998-11-02,NaN
2,1998-12-01,NaN
3,1999-01-01,NaN
4,1999-02-01,NaN
...,...,...
314,2025-01-02,20.200117
315,2025-02-03,21.619851
316,2025-03-03,21.689952
317,2025-04-01,22.063025


In [160]:
# Rename column headers
ftse250_volat.rename(columns={'time': 'date', 'HV': 'ftse250_volat'}, inplace=True)
ftse250_volat

,date,ftse250_volat
0,1998-10-01,NaN
1,1998-11-02,NaN
2,1998-12-01,NaN
3,1999-01-01,NaN
4,1999-02-01,NaN
...,...,...
314,2025-01-02,20.200117
315,2025-02-03,21.619851
316,2025-03-03,21.689952
317,2025-04-01,22.063025


In [161]:
# Change date to datetime datatype
ftse250_volat['date'] = pd.to_datetime(ftse250_volat['date'], format='%Y-%m-%d', dayfirst=True)
ftse250_volat

,date,ftse250_volat
0,1998-10-01,NaN
1,1998-11-02,NaN
2,1998-12-01,NaN
3,1999-01-01,NaN
4,1999-02-01,NaN
...,...,...
314,2025-01-02,20.200117
315,2025-02-03,21.619851
316,2025-03-03,21.689952
317,2025-04-01,22.063025


In [162]:
# Change date to monthly period type
ftse250_volat['date'] = ftse250_volat['date'].dt.to_period(freq="M")
ftse250_volat

,date,ftse250_volat
0,1998-10,NaN
1,1998-11,NaN
2,1998-12,NaN
3,1999-01,NaN
4,1999-02,NaN
...,...,...
314,2025-01,20.200117
315,2025-02,21.619851
316,2025-03,21.689952
317,2025-04,22.063025


In [163]:
validate_data(ftse250_volat)

Missing values per column:
date              0
ftse250_volat    10
dtype: int64

Unique values per column:
date             319
ftse250_volat    309
dtype: int64

Number of duplicate rows:
0

Summary statistics:
       ftse250_volat
count     309.000000
mean       30.325100
std        13.123221
min         9.863562
25%        21.472783
50%        27.475769
75%        34.157799
max        78.238668

Column has no zero values?
date             True
ftse250_volat    True
dtype: bool


In [164]:
# Save to csv
ftse250_volat.to_csv('ftse250_volat_clean.csv', index=False)

### FTSE All Share Historical Volatility

Source: https://www.tradingview.com/chart/rqC104t4/?symbol=FTSE%3AASX

In [165]:
# Import .csv
ftse_all_share_volat = pd.read_csv('ftse_all_share_volat.csv')
ftse_all_share_volat

,time,open,high,low,close,HV
0,2001-12-03,2532.6899,2538.8799,2518.8899,2523.8799,NaN
1,2002-01-02,2523.8601,2595.3101,2471.8301,2483.8101,NaN
2,2002-02-01,2496.2200,2524.0300,2424.5801,2466.9800,NaN
3,2002-03-01,2467.0100,2577.7500,2467.0100,2557.3999,NaN
4,2002-04-02,2557.4600,2567.7500,2494.0300,2512.0400,NaN
...,...,...,...,...,...,...
277,2025-01-02,4467.8000,4717.0200,4448.2700,4710.5800,16.441210
278,2025-02-03,4710.5800,4778.9200,4627.3600,4754.3200,16.158846
279,2025-03-03,4754.3200,4803.8900,4584.0900,4623.6200,17.468155
280,2025-04-01,4623.6200,4669.6600,4074.6800,4594.0500,17.145201


In [166]:
# Drop columns
ftse_all_share_volat = ftse_all_share_volat.drop(columns=['open', 'high', 'low', 'close'])
ftse_all_share_volat

,time,HV
0,2001-12-03,NaN
1,2002-01-02,NaN
2,2002-02-01,NaN
3,2002-03-01,NaN
4,2002-04-02,NaN
...,...,...
277,2025-01-02,16.441210
278,2025-02-03,16.158846
279,2025-03-03,17.468155
280,2025-04-01,17.145201


In [167]:
# Rename column headers
ftse_all_share_volat.rename(columns={'time': 'date', 'HV': 'ftse_all_share_volat'}, inplace=True)
ftse_all_share_volat

,date,ftse_all_share_volat
0,2001-12-03,NaN
1,2002-01-02,NaN
2,2002-02-01,NaN
3,2002-03-01,NaN
4,2002-04-02,NaN
...,...,...
277,2025-01-02,16.441210
278,2025-02-03,16.158846
279,2025-03-03,17.468155
280,2025-04-01,17.145201


In [168]:
# Change date to datetime datatype
ftse_all_share_volat['date'] = pd.to_datetime(ftse_all_share_volat['date'], format='%Y-%m-%d', dayfirst=True)
ftse_all_share_volat

,date,ftse_all_share_volat
0,2001-12-03,NaN
1,2002-01-02,NaN
2,2002-02-01,NaN
3,2002-03-01,NaN
4,2002-04-02,NaN
...,...,...
277,2025-01-02,16.441210
278,2025-02-03,16.158846
279,2025-03-03,17.468155
280,2025-04-01,17.145201


In [169]:
# Change date to monthly period type
ftse_all_share_volat['date'] = ftse_all_share_volat['date'].dt.to_period(freq="M")
ftse_all_share_volat

,date,ftse_all_share_volat
0,2001-12,NaN
1,2002-01,NaN
2,2002-02,NaN
3,2002-03,NaN
4,2002-04,NaN
...,...,...
277,2025-01,16.441210
278,2025-02,16.158846
279,2025-03,17.468155
280,2025-04,17.145201


In [170]:
validate_data(ftse_all_share_volat)

Missing values per column:
date                     0
ftse_all_share_volat    10
dtype: int64

Unique values per column:
date                    282
ftse_all_share_volat    272
dtype: int64

Number of duplicate rows:
0

Summary statistics:
       ftse_all_share_volat
count            272.000000
mean              24.315000
std               10.468104
min                9.900894
25%               15.884268
50%               21.963173
75%               29.556998
max               54.555319

Column has no zero values?
date                    True
ftse_all_share_volat    True
dtype: bool


**Note: there are 10 months for which historical volatility data is not provided. These missing values may cause issues if used in predictive modeling.**

In [171]:
# Save to csv
ftse_all_share_volat.to_csv('ftse_all_share_volat_clean.csv', index=False)

### Gilt Yields: 2-year and 10-year

In [172]:
# Import .csv
gilt_yields_2_10 = pd.read_csv('gilt_yields_2_10_clean.csv')
gilt_yields_2_10

,date,2_year,10_year
0,2005-01-04,4.37,4.48
1,2005-01-05,4.38,4.50
2,2005-01-06,4.34,4.47
3,2005-01-07,4.35,4.46
4,2005-01-10,4.34,4.45
...,...,...,...
5110,2025-03-25,4.08,4.73
5111,2025-03-26,4.06,4.70
5112,2025-03-27,4.06,4.77
5113,2025-03-28,4.00,4.69


In [173]:
# Rename columns
gilt_yields_2_10.rename(columns={'2_year': 'gilt_2y', '10_year': 'gilt_10y'}, inplace=True)
gilt_yields_2_10

,date,gilt_2y,gilt_10y
0,2005-01-04,4.37,4.48
1,2005-01-05,4.38,4.50
2,2005-01-06,4.34,4.47
3,2005-01-07,4.35,4.46
4,2005-01-10,4.34,4.45
...,...,...,...
5110,2025-03-25,4.08,4.73
5111,2025-03-26,4.06,4.70
5112,2025-03-27,4.06,4.77
5113,2025-03-28,4.00,4.69


In [174]:
# Change date to daily period data type
gilt_yields_2_10['date'] = pd.to_datetime(gilt_yields_2_10['date']).dt.to_period('D')
gilt_yields_2_10

,date,gilt_2y,gilt_10y
0,2005-01-04,4.37,4.48
1,2005-01-05,4.38,4.50
2,2005-01-06,4.34,4.47
3,2005-01-07,4.35,4.46
4,2005-01-10,4.34,4.45
...,...,...,...
5110,2025-03-25,4.08,4.73
5111,2025-03-26,4.06,4.70
5112,2025-03-27,4.06,4.77
5113,2025-03-28,4.00,4.69


In [175]:
validate_data(gilt_yields_2_10)

Missing values per column:
date        0
gilt_2y     0
gilt_10y    0
dtype: int64

Unique values per column:
date        5115
gilt_2y      522
gilt_10y     524
dtype: int64

Number of duplicate rows:
0

Summary statistics:
           gilt_2y     gilt_10y
count  5115.000000  5115.000000
mean      1.846608     2.747550
std       1.844931     1.461697
min      -0.140000     0.110000
25%       0.420000     1.440000
50%       0.820000     2.730000
75%       4.040000     4.150000
max       5.810000     5.440000

Column has no zero values?
date         True
gilt_2y     False
gilt_10y     True
dtype: bool


In [176]:
# Save to csv
gilt_yields_2_10.to_csv('gilt_2_10_clean.csv', index=False)

### OIS

In [177]:
# Import .csv
ois = pd.read_csv('ois_spot_clean.csv')
ois

,date,1_month,3_month,6_month,12_month
0,2016-01-04,0.46,0.47,0.49,0.56
1,2016-01-05,0.46,0.47,0.48,0.54
2,2016-01-06,0.46,0.47,0.48,0.53
3,2016-01-07,0.47,0.47,0.47,0.51
4,2016-01-08,0.46,0.47,0.47,0.50
...,...,...,...,...,...
2351,2025-04-24,4.32,4.17,4.01,3.78
2352,2025-04-25,4.31,4.17,4.01,3.78
2353,2025-04-28,4.29,4.16,4.00,3.77
2354,2025-04-29,4.28,4.16,3.99,3.75


In [178]:
# Rename columns
ois.rename(columns={'1_month': 'ois_1m', '3_month': 'ois_3m',
                                '6_month': 'ois_6m', '12_month': 'ois_12m'}, inplace=True)
ois

,date,ois_1m,ois_3m,ois_6m,ois_12m
0,2016-01-04,0.46,0.47,0.49,0.56
1,2016-01-05,0.46,0.47,0.48,0.54
2,2016-01-06,0.46,0.47,0.48,0.53
3,2016-01-07,0.47,0.47,0.47,0.51
4,2016-01-08,0.46,0.47,0.47,0.50
...,...,...,...,...,...
2351,2025-04-24,4.32,4.17,4.01,3.78
2352,2025-04-25,4.31,4.17,4.01,3.78
2353,2025-04-28,4.29,4.16,4.00,3.77
2354,2025-04-29,4.28,4.16,3.99,3.75


In [179]:
# Change date to daily period data type
ois['date'] = pd.to_datetime(ois['date']).dt.to_period('D')
ois

,date,ois_1m,ois_3m,ois_6m,ois_12m
0,2016-01-04,0.46,0.47,0.49,0.56
1,2016-01-05,0.46,0.47,0.48,0.54
2,2016-01-06,0.46,0.47,0.48,0.53
3,2016-01-07,0.47,0.47,0.47,0.51
4,2016-01-08,0.46,0.47,0.47,0.50
...,...,...,...,...,...
2351,2025-04-24,4.32,4.17,4.01,3.78
2352,2025-04-25,4.31,4.17,4.01,3.78
2353,2025-04-28,4.29,4.16,4.00,3.77
2354,2025-04-29,4.28,4.16,3.99,3.75


In [180]:
validate_data(ois)

Missing values per column:
date       0
ois_1m     0
ois_3m     0
ois_6m     0
ois_12m    0
dtype: int64

Unique values per column:
date       2356
ois_1m      329
ois_3m      342
ois_6m      343
ois_12m     379
dtype: int64

Number of duplicate rows:
0

Summary statistics:
            ois_1m       ois_3m       ois_6m      ois_12m
count  2356.000000  2356.000000  2356.000000  2356.000000
mean      1.594537     1.635862     1.680497     1.715866
std       1.940598     1.950987     1.958909     1.935753
min       0.040000     0.020000     0.000000    -0.040000
25%       0.210000     0.210000     0.220000     0.250000
50%       0.670000     0.670000     0.650000     0.655000
75%       3.450000     3.730000     4.060000     4.140000
max       5.370000     5.430000     5.690000     6.080000

Column has no zero values?
date        True
ois_1m      True
ois_3m      True
ois_6m     False
ois_12m    False
dtype: bool


In [181]:
# Save to csv
ois.to_csv('ois_clean.csv', index=False)

### GfK Consumer Confidence

Source: https://tradingeconomics.com/united-kingdom/consumer-confidence

In [182]:
# Import .csv
gfk_cc = pd.read_csv('gfk_consumer_confidence_clean.csv')
gfk_cc

,date,gfk_cc
0,2025-04-24,-23
1,2025-03-20,-19
2,2025-02-20,-20
3,2025-01-23,-22
4,2024-12-12,-17
...,...,...
318,1998-08-01,-1
319,1998-07-01,2
320,1998-06-01,7
321,1998-05-01,4


In [183]:
# Change date to monthly period data type
gfk_cc['date'] = pd.to_datetime(gfk_cc['date']).dt.to_period('M')
gfk_cc

,date,gfk_cc
0,2025-04,-23
1,2025-03,-19
2,2025-02,-20
3,2025-01,-22
4,2024-12,-17
...,...,...
318,1998-08,-1
319,1998-07,2
320,1998-06,7
321,1998-05,4


In [184]:
validate_data(gfk_cc)

Missing values per column:
date      0
gfk_cc    0
dtype: int64

Unique values per column:
date      323
gfk_cc     55
dtype: int64

Number of duplicate rows:
0

Summary statistics:
           gfk_cc
count  323.000000
mean   -12.167183
std     12.739260
min    -49.000000
25%    -21.000000
50%     -9.000000
75%     -2.000000
max      8.000000

Column has no zero values?
date       True
gfk_cc    False
dtype: bool


In [185]:
# Save to csv
gfk_cc.to_csv('gfk_cc_clean.csv', index=False)

### House Price Index (Month-over-Month)

Source: https://tradingeconomics.com/united-kingdom/house-price-index-mom

In [186]:
# Import .csv
hpi_mom = pd.read_csv('house_price_index_mom_clean.csv')
hpi_mom

,date,house_price_index_mom
0,1983-02-28,0.8
1,1983-03-31,0.7
2,1983-04-30,0.8
3,1983-05-31,1.4
4,1983-06-30,0.2
...,...,...
501,2024-11-30,1.2
502,2024-12-31,-0.2
503,2025-01-31,0.6
504,2025-02-28,-0.2


In [187]:
# Rename columns
hpi_mom.rename(columns={'house_price_index_mom': 'hpi_mom'}, inplace=True)
hpi_mom

,date,hpi_mom
0,1983-02-28,0.8
1,1983-03-31,0.7
2,1983-04-30,0.8
3,1983-05-31,1.4
4,1983-06-30,0.2
...,...,...
501,2024-11-30,1.2
502,2024-12-31,-0.2
503,2025-01-31,0.6
504,2025-02-28,-0.2


In [188]:
# Change date to monthly period data type
hpi_mom['date'] = pd.to_datetime(hpi_mom['date']).dt.to_period('M')
hpi_mom

,date,hpi_mom
0,1983-02,0.8
1,1983-03,0.7
2,1983-04,0.8
3,1983-05,1.4
4,1983-06,0.2
...,...,...
501,2024-11,1.2
502,2024-12,-0.2
503,2025-01,0.6
504,2025-02,-0.2


In [189]:
# Save to csv
hpi_mom.to_csv('hpi_mom_clean.csv', index=False)

### House Price Index (Year-over-Year)

Source: https://tradingeconomics.com/united-kingdom/house-price-index-yoy

In [190]:
# Import .csv
hpi_yoy = pd.read_csv('house_price_index_yoy_clean.csv')
hpi_yoy

,date,house_price_index_yoy
0,1984-01-31,7.3
1,1984-02-29,7.1
2,1984-03-31,7.3
3,1984-04-30,6.4
4,1984-05-31,6.1
...,...,...
490,2024-11-30,4.7
491,2024-12-31,3.4
492,2025-01-31,2.9
493,2025-02-28,2.8


In [191]:
# Rename columns
hpi_yoy.rename(columns={'house_price_index_yoy': 'hpi_yoy'}, inplace=True)
hpi_yoy

,date,hpi_yoy
0,1984-01-31,7.3
1,1984-02-29,7.1
2,1984-03-31,7.3
3,1984-04-30,6.4
4,1984-05-31,6.1
...,...,...
490,2024-11-30,4.7
491,2024-12-31,3.4
492,2025-01-31,2.9
493,2025-02-28,2.8


In [192]:
# Change date to monthly period data type
hpi_yoy['date'] = pd.to_datetime(hpi_yoy['date']).dt.to_period('M')
hpi_yoy

,date,hpi_yoy
0,1984-01,7.3
1,1984-02,7.1
2,1984-03,7.3
3,1984-04,6.4
4,1984-05,6.1
...,...,...
490,2024-11,4.7
491,2024-12,3.4
492,2025-01,2.9
493,2025-02,2.8


In [193]:
validate_data(hpi_yoy)

Missing values per column:
date       0
hpi_yoy    0
dtype: int64

Unique values per column:
date       495
hpi_yoy    227
dtype: int64

Number of duplicate rows:
0

Summary statistics:
          hpi_yoy
count  495.000000
mean     6.231717
std      7.957991
min    -16.800000
25%      1.500000
50%      6.000000
75%      9.600000
max     34.500000

Column has no zero values?
date        True
hpi_yoy    False
dtype: bool


In [194]:
# Save to csv
hpi_yoy.to_csv('hpi_yoy_clean.csv', index=False)

### House Prices

Source: https://tradingeconomics.com/united-kingdom/housing-index

In [195]:
# Import .csv
house_prices = pd.read_csv('house_prices_avg_monthly_clean.csv')
house_prices

,date,Close
0,1983-01-31,26187.81
1,1983-02-28,26411.08
2,1983-03-31,26603.62
3,1983-04-30,26823.41
4,1983-05-31,27205.58
...,...,...
502,2024-11-30,297829.00
503,2024-12-31,297179.00
504,2025-01-31,298815.00
505,2025-02-28,298274.00


In [196]:
# Rename columns
house_prices.rename(columns={'Close': 'avg_house_price'}, inplace=True)
house_prices

,date,avg_house_price
0,1983-01-31,26187.81
1,1983-02-28,26411.08
2,1983-03-31,26603.62
3,1983-04-30,26823.41
4,1983-05-31,27205.58
...,...,...
502,2024-11-30,297829.00
503,2024-12-31,297179.00
504,2025-01-31,298815.00
505,2025-02-28,298274.00


In [197]:
# Change date to monthly period data type
house_prices['date'] = pd.to_datetime(house_prices['date']).dt.to_period('M')
house_prices

,date,avg_house_price
0,1983-01,26187.81
1,1983-02,26411.08
2,1983-03,26603.62
3,1983-04,26823.41
4,1983-05,27205.58
...,...,...
502,2024-11,297829.00
503,2024-12,297179.00
504,2025-01,298815.00
505,2025-02,298274.00


In [198]:
validate_data(house_prices)

Missing values per column:
date               0
avg_house_price    0
dtype: int64

Unique values per column:
date               507
avg_house_price    505
dtype: int64

Number of duplicate rows:
0

Summary statistics:
       avg_house_price
count       507.000000
mean     136087.283176
std       81818.395056
min       26187.810000
25%       59096.405000
50%      146572.170000
75%      193788.450000
max      298815.000000

Column has no zero values?
date               True
avg_house_price    True
dtype: bool


In [199]:
# Save to csv
house_prices.to_csv('avg_house_price_clean.csv', index=False)

### Mortgage Approvals

Source: https://tradingeconomics.com/united-kingdom/mortgage-approvals

In [200]:
# Import .csv
mortg_apprv = pd.read_csv('mortgage_approvals_clean.csv')
mortg_apprv

,date,mortgage_approvals
0,1986-10-31,120.800
1,1986-11-30,115.800
2,1986-12-31,120.800
3,1987-01-31,118.400
4,1987-02-28,111.400
...,...,...
457,2024-11-30,66.076
458,2024-12-31,66.505
459,2025-01-31,66.040
460,2025-02-28,65.090


In [201]:
# Change date to monthly period data type
mortg_apprv['date'] = pd.to_datetime(mortg_apprv['date']).dt.to_period('M')
mortg_apprv

,date,mortgage_approvals
0,1986-10,120.800
1,1986-11,115.800
2,1986-12,120.800
3,1987-01,118.400
4,1987-02,111.400
...,...,...
457,2024-11,66.076
458,2024-12,66.505
459,2025-01,66.040
460,2025-02,65.090


In [202]:
validate_data(mortg_apprv)

Missing values per column:
date                  0
mortgage_approvals    0
dtype: int64

Unique values per column:
date                  462
mortgage_approvals    452
dtype: int64

Number of duplicate rows:
0

Summary statistics:
       mortgage_approvals
count          462.000000
mean            81.301567
std             25.001032
min              9.356000
25%             64.921250
50%             79.796500
75%             98.621000
max            151.800000

Column has no zero values?
date                  True
mortgage_approvals    True
dtype: bool


In [203]:
# Save to csv
mortg_apprv.to_csv('mortg_apprv_clean.csv', index=False)

### Loans to Private Sector

Source: https://tradingeconomics.com/united-kingdom/loans-to-private-sector

In [204]:
# Import .csv
prvt_sctr_lns = pd.read_csv('loans_to_private_sector_clean.csv')
prvt_sctr_lns

,date,loans_to_pvt_sector
0,1963-03-31,8755
1,1963-06-30,9046
2,1963-09-30,9307
3,1963-12-31,9592
4,1964-03-31,9890
...,...,...
243,2023-12-31,2694208
244,2024-03-31,2704587
245,2024-06-30,2710539
246,2024-09-30,2735199


In [205]:
# Change date to monthly period data type
prvt_sctr_lns['date'] = pd.to_datetime(prvt_sctr_lns['date']).dt.to_period('Q')
prvt_sctr_lns

,date,loans_to_pvt_sector
0,1963Q1,8755
1,1963Q2,9046
2,1963Q3,9307
3,1963Q4,9592
4,1964Q1,9890
...,...,...
243,2023Q4,2694208
244,2024Q1,2704587
245,2024Q2,2710539
246,2024Q3,2735199


In [206]:
validate_data(prvt_sctr_lns)

Missing values per column:
date                   0
loans_to_pvt_sector    0
dtype: int64

Unique values per column:
date                   248
loans_to_pvt_sector    248
dtype: int64

Number of duplicate rows:
0

Summary statistics:
       loans_to_pvt_sector
count         2.480000e+02
mean          1.041013e+06
std           1.020466e+06
min           8.755000e+03
25%           6.525275e+04
50%           6.416135e+05
75%           2.195002e+06
max           2.812956e+06

Column has no zero values?
date                   True
loans_to_pvt_sector    True
dtype: bool


In [207]:
# Save to csv
prvt_sctr_lns.to_csv('private_sector_loans_clean.csv', index=False)

### Business Confidence

Source: https://tradingeconomics.com/united-kingdom/business-confidence

In [208]:
# Import .csv
biz_conf = pd.read_csv('business_confidence_clean.csv')
biz_conf

,date,business_confidence
0,1958-09-30,-12
1,1958-12-31,18
2,1959-03-31,20
3,1959-09-30,55
4,1959-12-31,48
...,...,...
249,2024-06-30,9
250,2024-09-30,-9
251,2024-12-31,-24
252,2025-03-31,-47


In [209]:
# Change date to quarterly period data type
biz_conf['date'] = pd.to_datetime(biz_conf['date']).dt.to_period('Q')
biz_conf

,date,business_confidence
0,1958Q3,-12
1,1958Q4,18
2,1959Q1,20
3,1959Q3,55
4,1959Q4,48
...,...,...
249,2024Q2,9
250,2024Q3,-9
251,2024Q4,-24
252,2025Q1,-47


In [210]:
validate_data(biz_conf)

Missing values per column:
date                   0
business_confidence    0
dtype: int64

Unique values per column:
date                   254
business_confidence     92
dtype: int64

Number of duplicate rows:
0

Summary statistics:
       business_confidence
count           254.000000
mean             -4.511811
std              24.197431
min             -87.000000
25%             -18.000000
50%              -3.000000
75%              10.000000
max              55.000000

Column has no zero values?
date                    True
business_confidence    False
dtype: bool


In [211]:
# Save to csv
biz_conf.to_csv('biz_conf_clean.csv', index=False)

### Consumer Spending

Source: https://tradingeconomics.com/united-kingdom/consumer-spending

In [212]:
# Import .csv
cons_spend = pd.read_csv('consumer_spending_clean.csv')
cons_spend

,date,consumer_spending
0,1955-03-31,74229
1,1955-06-30,74710
2,1955-09-30,75770
3,1955-12-31,75291
4,1956-03-31,75511
...,...,...
275,2023-12-31,384581
276,2024-03-31,387140
277,2024-06-30,386912
278,2024-09-30,388728


In [213]:
# Change date to monthly period data type
cons_spend['date'] = pd.to_datetime(cons_spend['date']).dt.to_period('Q')
cons_spend

,date,consumer_spending
0,1955Q1,74229
1,1955Q2,74710
2,1955Q3,75770
3,1955Q4,75291
4,1956Q1,75511
...,...,...
275,2023Q4,384581
276,2024Q1,387140
277,2024Q2,386912
278,2024Q3,388728


In [214]:
validate_data(cons_spend)

Missing values per column:
date                 0
consumer_spending    0
dtype: int64

Unique values per column:
date                 280
consumer_spending    280
dtype: int64

Number of duplicate rows:
0

Summary statistics:
       consumer_spending
count         280.000000
mean       212054.010714
std        103057.056427
min         74229.000000
25%        122829.500000
50%        199065.500000
75%        314627.000000
max        389219.000000

Column has no zero values?
date                 True
consumer_spending    True
dtype: bool


In [215]:
# Save to csv
cons_spend.to_csv('consumer_spend_clean.csv', index=False)

### Additional Quarterly Indicators

house price quarterly change, GfK consumer confidence and inflation expectations (monthly to quarterly conversions)

In [216]:
# Import .csv
add_quarterly = pd.read_csv('add_quarterly.csv')
add_quarterly

,quarter,cpi_rate,gdp_rate,household_debt,loans_to_pvt_sector,unemp_rate,business_confidence,consumer_spending,wage_rate,rsi_mom,rsi_3mo3m,rsi_3mo3m1y,rsi_vol,house_price_change,gfk_avg_cc_qrt,infl_exp_qrt
0,1955Q1,NaN,NaN,NaN,NaN,NaN,NaN,74229.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1955Q2,NaN,0.0,NaN,NaN,NaN,NaN,74710.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1955Q3,NaN,2.0,NaN,NaN,NaN,NaN,75770.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1955Q4,NaN,-0.6,NaN,NaN,NaN,NaN,75291.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1956Q1,NaN,1.1,NaN,NaN,NaN,NaN,75511.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,2024Q2,2.1,0.5,121.0,2710539.0,4.2,9.0,386912.0,5.73,NaN,-0.5,NaN,97.1,0.00,-16.67,3.0
278,2024Q3,2.0,0.0,120.2,2735199.0,4.3,-9.0,388728.0,5.17,NaN,1.3,NaN,98.3,1.51,-15.33,NaN
279,2024Q4,2.5,0.1,118.1,2715714.0,4.4,-24.0,389219.0,5.63,NaN,-0.7,NaN,97.6,1.40,-18.67,NaN
280,2025Q1,2.8,NaN,NaN,NaN,NaN,-47.0,NaN,5.85,NaN,1.6,NaN,99.2,-0.10,-20.33,NaN


In [217]:
add_quarterly.dtypes

quarter                 object
cpi_rate               float64
gdp_rate               float64
household_debt         float64
loans_to_pvt_sector    float64
unemp_rate             float64
business_confidence    float64
consumer_spending      float64
wage_rate              float64
rsi_mom                float64
rsi_3mo3m              float64
rsi_3mo3m1y            float64
rsi_vol                float64
house_price_change     float64
gfk_avg_cc_qrt         float64
infl_exp_qrt           float64
dtype: object

In [218]:
# Rename columns
add_quarterly.rename(columns={'quarter': 'date'}, inplace=True)
add_quarterly

,date,cpi_rate,gdp_rate,household_debt,loans_to_pvt_sector,unemp_rate,business_confidence,consumer_spending,wage_rate,rsi_mom,rsi_3mo3m,rsi_3mo3m1y,rsi_vol,house_price_change,gfk_avg_cc_qrt,infl_exp_qrt
0,1955Q1,NaN,NaN,NaN,NaN,NaN,NaN,74229.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1955Q2,NaN,0.0,NaN,NaN,NaN,NaN,74710.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1955Q3,NaN,2.0,NaN,NaN,NaN,NaN,75770.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1955Q4,NaN,-0.6,NaN,NaN,NaN,NaN,75291.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1956Q1,NaN,1.1,NaN,NaN,NaN,NaN,75511.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,2024Q2,2.1,0.5,121.0,2710539.0,4.2,9.0,386912.0,5.73,NaN,-0.5,NaN,97.1,0.00,-16.67,3.0
278,2024Q3,2.0,0.0,120.2,2735199.0,4.3,-9.0,388728.0,5.17,NaN,1.3,NaN,98.3,1.51,-15.33,NaN
279,2024Q4,2.5,0.1,118.1,2715714.0,4.4,-24.0,389219.0,5.63,NaN,-0.7,NaN,97.6,1.40,-18.67,NaN
280,2025Q1,2.8,NaN,NaN,NaN,NaN,-47.0,NaN,5.85,NaN,1.6,NaN,99.2,-0.10,-20.33,NaN


In [219]:
add_quarterly = add_quarterly.loc[:, ['date', 'house_price_change',	'gfk_avg_cc_qrt', 'infl_exp_qrt']]
add_quarterly

,date,house_price_change,gfk_avg_cc_qrt,infl_exp_qrt
0,1955Q1,NaN,NaN,NaN
1,1955Q2,NaN,NaN,NaN
2,1955Q3,NaN,NaN,NaN
3,1955Q4,NaN,NaN,NaN
4,1956Q1,NaN,NaN,NaN
...,...,...,...,...
277,2024Q2,0.00,-16.67,3.0
278,2024Q3,1.51,-15.33,NaN
279,2024Q4,1.40,-18.67,NaN
280,2025Q1,-0.10,-20.33,NaN


In [220]:
add_quarterly['date'] = pd.to_datetime(add_quarterly['date']).dt.to_period('Q')
add_quarterly

/var/folders/hk/dv8kkk0x06n17jn62mhyd1cw0000gn/T/ipykernel_37434/3359795766.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  add_quarterly['date'] = pd.to_datetime(add_quarterly['date']).dt.to_period('Q')


,date,house_price_change,gfk_avg_cc_qrt,infl_exp_qrt
0,1955Q1,NaN,NaN,NaN
1,1955Q2,NaN,NaN,NaN
2,1955Q3,NaN,NaN,NaN
3,1955Q4,NaN,NaN,NaN
4,1956Q1,NaN,NaN,NaN
...,...,...,...,...
277,2024Q2,0.00,-16.67,3.0
278,2024Q3,1.51,-15.33,NaN
279,2024Q4,1.40,-18.67,NaN
280,2025Q1,-0.10,-20.33,NaN


In [221]:
# Check data types
add_quarterly.dtypes

date                  period[Q-DEC]
house_price_change          float64
gfk_avg_cc_qrt              float64
infl_exp_qrt                float64
dtype: object

## Join Daily Indicators

In [222]:
# Join using reduce with merge
dfs = [ftse250, ftse100, gilt_yields_2_10, ois, sonia, vix, gbp_usd_daily]

# Join all DataFrames using reduce with an outer merge on the date column
daily_indicators = reduce(lambda left, right: pd.merge(left, right, on='date', how='outer'), dfs)

# Sort by date for clarity
daily_indicators = daily_indicators.sort_values('date').reset_index(drop=True)

daily_indicators

,date,ftse250,ftse100,gilt_2y,gilt_10y,ois_1m,ois_3m,ois_6m,ois_12m,sonia,vix,gbp_usd_d
0,1986-03-04,NaN,1548.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1986-03-05,NaN,1569.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1986-03-06,NaN,1566.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1986-03-07,NaN,1573.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1986-03-10,NaN,1572.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
10220,2025-05-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.64,1.3284
10221,2025-05-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.76,1.3369
10222,2025-05-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.55,1.3344
10223,2025-05-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.3287


In [223]:
validate_data(daily_indicators)

Missing values per column:
date            0
ftse250      1297
ftse100       225
gilt_2y      5110
gilt_10y     5110
ois_1m       7869
ois_3m       7869
ois_6m       7869
ois_12m      7869
sonia        3067
vix          1303
gbp_usd_d    3112
dtype: int64

Unique values per column:
date         10225
ftse250       8886
ftse100       9591
gilt_2y        522
gilt_10y       524
ois_1m         329
ois_3m         342
ois_6m         343
ois_12m        379
sonia         4606
vix           2521
gbp_usd_d     4318
dtype: int64

Number of duplicate rows:
0

Summary statistics:
            ftse250       ftse100      gilt_2y     gilt_10y       ois_1m  \
count   8928.000000  10000.000000  5115.000000  5115.000000  2356.000000   
mean   10595.070865   5119.341922     1.846608     2.747550     1.594537   
std     6485.312480   1887.976603     1.844931     1.461697     1.940598   
min     1967.570000   1526.700000    -0.140000     0.110000     0.040000   
25%     4742.427500   3522.625000     0.420000

**There are unique date values for every row.**

In [224]:
# Save to csv
daily_indicators.to_csv('daily_indicators.csv', index=False)

## Join Monthly Indicators

In [225]:
# Join using reduce with merge
dfs = [cpi_monthly, ftse_vix_monthly, gbp_usd_monthly, gfk_cc, hpi_mom, hpi_yoy,
       house_prices, infl_exp, mortg_apprv, mortgage_rates, rsi_monthly, unemp_monthly,
       wage_monthly, pmi_monthly, ftse100_volat, ftse250_monthly, ftse250_volat,
       ftse_all_share_monthly, ftse_all_share_volat, gdp_monthly, household_debt_monthly]

# Join all DataFrames using reduce with an outer merge on the date column
monthly_indicators = reduce(lambda left, right: pd.merge(left, right, on='date', how='outer'), dfs)

# Sort by date for clarity
monthly_indicators = monthly_indicators.sort_values('date').reset_index(drop=True)

monthly_indicators

,date,cpi_rate,ftse_vix,gbp_usd_m,gfk_cc,hpi_mom,hpi_yoy,avg_house_price,infl_exp,mortgage_approvals,...,wage_rate,pmi,ftse100_monthly,ftse100_volat,ftse250_monthly,ftse250_volat,ftse_all_share_monthly,ftse_all_share_volat,gdp_rate,household_debt
0,1955-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1,1955-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2,1955-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
3,1955-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
4,1955-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
837,2025-01,3.0,NaN,1.2348,-22.0,0.6,2.9,298815.0,3.5,66.04,...,5.8,50.65,8673.96,17.061888,20950.48,20.200117,"4,710.58",16.441210,NaN,NaN
838,2025-02,2.8,NaN,1.2545,-20.0,-0.2,2.8,298274.0,3.9,65.09,...,5.9,50.55,8809.74,16.771649,20326.38,21.619851,"4,754.32",16.158846,NaN,NaN
839,2025-03,2.6,NaN,1.2911,-19.0,-0.5,2.8,296699.0,NaN,64.31,...,NaN,51.25,8582.81,18.130499,19475.48,21.689952,"4,623.62",17.468155,NaN,NaN
840,2025-04,NaN,NaN,1.3131,-23.0,NaN,NaN,NaN,NaN,NaN,...,NaN,49.85,8494.85,17.983591,19884.59,22.063025,"4,594.05",17.145201,NaN,NaN


In [226]:
# Rename columns
monthly_indicators.rename(columns={'date': 'month'}, inplace=True)
monthly_indicators

,month,cpi_rate,ftse_vix,gbp_usd_m,gfk_cc,hpi_mom,hpi_yoy,avg_house_price,infl_exp,mortgage_approvals,...,wage_rate,pmi,ftse100_monthly,ftse100_volat,ftse250_monthly,ftse250_volat,ftse_all_share_monthly,ftse_all_share_volat,gdp_rate,household_debt
0,1955-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1,1955-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2,1955-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
3,1955-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
4,1955-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
837,2025-01,3.0,NaN,1.2348,-22.0,0.6,2.9,298815.0,3.5,66.04,...,5.8,50.65,8673.96,17.061888,20950.48,20.200117,"4,710.58",16.441210,NaN,NaN
838,2025-02,2.8,NaN,1.2545,-20.0,-0.2,2.8,298274.0,3.9,65.09,...,5.9,50.55,8809.74,16.771649,20326.38,21.619851,"4,754.32",16.158846,NaN,NaN
839,2025-03,2.6,NaN,1.2911,-19.0,-0.5,2.8,296699.0,NaN,64.31,...,NaN,51.25,8582.81,18.130499,19475.48,21.689952,"4,623.62",17.468155,NaN,NaN
840,2025-04,NaN,NaN,1.3131,-23.0,NaN,NaN,NaN,NaN,NaN,...,NaN,49.85,8494.85,17.983591,19884.59,22.063025,"4,594.05",17.145201,NaN,NaN


In [227]:
validate_data(monthly_indicators)

Missing values per column:
month                       0
cpi_rate                  407
ftse_vix                  663
gbp_usd_m                 238
gfk_cc                    519
hpi_mom                   336
hpi_yoy                   347
avg_house_price           335
infl_exp                  588
mortgage_approvals        380
mortgage_2yr              478
mortgage_svr              478
rsi_mom                   492
rsi_3mo3m                 496
rsi_3mo3m1y               505
rsi_vol                   491
unemp_rate                194
wage_rate                 554
pmi                       773
ftse100_monthly           527
ftse100_volat             527
ftse250_monthly           717
ftse250_volat             533
ftse_all_share_monthly    571
ftse_all_share_volat      570
gdp_rate                    5
household_debt            386
dtype: int64

Unique values per column:
month                     842
cpi_rate                   86
ftse_vix                  168
gbp_usd_m                 576
gfk

In [228]:
# Save to csv
monthly_indicators.to_csv('monthly_indicators.csv', index=False)

## Join Quarterly Indicators

In [229]:
# Join using reduce with merge
dfs = [cpi_quarterly, gdp, household_debt, prvt_sctr_lns, unemp_quarterly,
       biz_conf, cons_spend, wage_quarterly, rsi_quarterly, add_quarterly]

# Join all DataFrames using reduce with an outer merge on the date column
quarterly_indicators = reduce(lambda left, right: pd.merge(left, right, on='date', how='outer'), dfs)

# Sort by date for clarity
quarterly_indicators = quarterly_indicators.sort_values('date').reset_index(drop=True)

print(quarterly_indicators.head())

     date  cpi_rate  gdp_rate  household_debt  loans_to_pvt_sector  \
0  1955Q1       NaN       NaN             NaN                  NaN   
1  1955Q2       NaN       0.0             NaN                  NaN   
2  1955Q3       NaN       2.0             NaN                  NaN   
3  1955Q4       NaN      -0.6             NaN                  NaN   
4  1956Q1       NaN       1.1             NaN                  NaN   

   unemp_rate  business_confidence  consumer_spending  wage_rate  rsi_3mo3m  \
0         NaN                  NaN            74229.0        NaN        NaN   
1         NaN                  NaN            74710.0        NaN        NaN   
2         NaN                  NaN            75770.0        NaN        NaN   
3         NaN                  NaN            75291.0        NaN        NaN   
4         NaN                  NaN            75511.0        NaN        NaN   

   rsi_vol  house_price_change  gfk_avg_cc_qrt  infl_exp_qrt  
0      NaN                 NaN           

In [230]:
# Rename columns
quarterly_indicators.rename(columns={'date': 'quarter'}, inplace=True)
quarterly_indicators

,quarter,cpi_rate,gdp_rate,household_debt,loans_to_pvt_sector,unemp_rate,business_confidence,consumer_spending,wage_rate,rsi_3mo3m,rsi_vol,house_price_change,gfk_avg_cc_qrt,infl_exp_qrt
0,1955Q1,NaN,NaN,NaN,NaN,NaN,NaN,74229.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1955Q2,NaN,0.0,NaN,NaN,NaN,NaN,74710.0,NaN,NaN,NaN,NaN,NaN,NaN
2,1955Q3,NaN,2.0,NaN,NaN,NaN,NaN,75770.0,NaN,NaN,NaN,NaN,NaN,NaN
3,1955Q4,NaN,-0.6,NaN,NaN,NaN,NaN,75291.0,NaN,NaN,NaN,NaN,NaN,NaN
4,1956Q1,NaN,1.1,NaN,NaN,NaN,NaN,75511.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,2024Q2,2.1,0.5,121.0,2710539.0,4.2,9.0,386912.0,5.73,-0.5,97.1,0.00,-16.67,3.0
278,2024Q3,2.0,0.0,120.2,2735199.0,4.3,-9.0,388728.0,5.17,1.3,98.3,1.51,-15.33,NaN
279,2024Q4,2.5,0.1,118.1,2715714.0,4.4,-24.0,389219.0,5.63,-0.7,97.6,1.40,-18.67,NaN
280,2025Q1,2.8,NaN,NaN,NaN,NaN,-47.0,NaN,5.85,1.6,99.2,-0.10,-20.33,NaN


In [231]:
validate_data(quarterly_indicators)

Missing values per column:
quarter                  0
cpi_rate               137
gdp_rate                 3
household_debt         130
loans_to_pvt_sector     34
unemp_rate              66
business_confidence     28
consumer_spending        2
wage_rate              185
rsi_3mo3m              166
rsi_vol                165
house_price_change     114
gfk_avg_cc_qrt         174
infl_exp_qrt           208
dtype: int64

Unique values per column:
quarter                282
cpi_rate                60
gdp_rate                52
household_debt         129
loans_to_pvt_sector    248
unemp_rate              73
business_confidence     92
consumer_spending      280
wage_rate               74
rsi_3mo3m               44
rsi_vol                 99
house_price_change      85
gfk_avg_cc_qrt          67
infl_exp_qrt            52
dtype: int64

Number of duplicate rows:
0

Summary statistics:
         cpi_rate    gdp_rate  household_debt  loans_to_pvt_sector  \
count  145.000000  279.000000      152.00000

In [232]:
# Save to csv
quarterly_indicators.to_csv('quarterly_indicators.csv', index=False)

## Join Finbert and Cbroberta dataframes

In [233]:
# Import .csv file
cbroberta = pd.read_csv('agent_sentiment_pivot.csv')
cbroberta

,custom_ref,date,author,is_gov,central_bank,financial_sector,firms,government,households,cbroberta_avg_score,cbroberta_avg_sentiment,text
0,UK_1998-09-15_George,1998-09-15,Edward George,1,-0.409745,-0.661564,-0.202167,-0.518037,-0.042013,-0.295018,0,"Thank you, Chairman. I'm actually very pleased..."
1,UK_1998-10-21_George,1998-10-21,Edward George,1,-0.134019,0.047685,-0.305432,-0.251951,-0.535169,-0.128637,0,It's a great pleasure to be here in the beauti...
2,UK_1998-10-21_King,1998-10-21,Mervyn King,0,-0.516879,-0.644094,-0.581463,-0.580222,-0.672911,-0.601670,0,Few industries have suffered more from volatil...
3,UK_1998-11-01_budd,1998-11-01,budd,0,-0.497668,-0.373081,-0.598642,-0.567226,-0.360106,-0.476216,0,My topic this evening is the use of forecasts ...
4,UK_1998-11-01_vickers,1998-11-01,vickers,0,-0.555400,-0.629627,-0.683497,-0.339261,-0.289324,-0.569214,0,"Six years ago this week, sterling left the exc..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1360,UK_2025-02-28_Ramsden,2025-02-28,Dave Ramsden,0,-0.587502,-0.648696,-0.595713,-0.572169,-0.436912,-0.547658,0,Speech Thank you for the invitation to speak ...
1361,UK_2025-03-03_Kyei,2025-03-03,Afua Kyei,0,0.479018,0.512251,0.355660,0.497915,0.269962,0.421629,1,"Speech Good evening everyone, students and al..."
1362,UK_2025-03-06_Mann,2025-03-06,L. Mann,0,-0.567989,-0.744008,-0.746682,-0.258689,-0.682128,-0.634426,0,Speech Introduction It is particularly fitti...
1363,UK_2025-03-24_Bailey,2025-03-24,Andrew Bailey,1,-0.159905,-0.521657,0.081224,-0.194019,-0.030078,-0.046111,0,Speech Thank you for inviting me to speak tod...


In [234]:
# Import .csv file
finbert = pd.read_csv('uk_speeches_with_finbert_scores_v2.csv')
finbert

,date,custom_ref,reference,title,author,is_gov,text,finbert_pos,finbert_neg,finbert_neu,finbert_sentiment,finbert_score,finbert_confidence
0,1998-09-15,UK_1998-09-15_George,r980915a_BOE,Speech,Edward George,1.0,"Thank you, Chairman. I'm actually very pleased...",0.081732,0.281070,0.637198,neutral,-0.199338,0.637198
1,1998-10-21,UK_1998-10-21_George,r981021b_BOE,Britain in Europe,Edward George,1.0,It's a great pleasure to be here in the beauti...,0.153660,0.028016,0.818323,neutral,0.125644,0.818323
2,1998-10-21,UK_1998-10-21_King,r981021a_BOE,Impact of the recent turbulence in internation...,Mervyn King,0.0,Few industries have suffered more from volatil...,0.038727,0.808180,0.153093,negative,-0.769453,0.808180
3,1998-11-01,UK_1998-11-01_budd,r981101a_BOE,"Economic policy, with and without forecasts",budd,0.0,My topic this evening is the use of forecasts ...,0.062295,0.135857,0.801848,neutral,-0.073561,0.801848
4,1998-11-01,UK_1998-11-01_vickers,r981101b_BOE,Inflation targeting in practice: the UK experi...,vickers,0.0,"Six years ago this week, sterling left the exc...",0.082896,0.082629,0.834475,neutral,0.000268,0.834475
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1360,2025-02-28,UK_2025-02-28_Ramsden,NaN,"Surveys, forecasts and scenarios: setting UK m...",Dave Ramsden,0.0,Speech Thank you for the invitation to speak ...,0.087475,0.032670,0.879856,neutral,0.054805,0.879856
1361,2025-03-03,UK_2025-03-03_Kyei,NaN,"Accountants can influence growth, competitiven...",Afua Kyei,0.0,"Speech Good evening everyone, students and al...",0.051385,0.035248,0.913366,neutral,0.016137,0.913366
1362,2025-03-06,UK_2025-03-06_Mann,NaN,Holding the anchor in turbulent waters  speec...,L. Mann,0.0,Speech Introduction It is particularly fitti...,0.101165,0.022226,0.876609,neutral,0.078938,0.876609
1363,2025-03-24,UK_2025-03-24_Bailey,NaN,Growth: What does it take in todays world? - ...,Andrew Bailey,1.0,Speech Thank you for inviting me to speak tod...,0.059699,0.031643,0.908659,neutral,0.028056,0.908659


In [235]:
sentiments = pd.merge(finbert, cbroberta, on='custom_ref', suffixes=('', '_drop'))
sentiments = sentiments.loc[:, ~sentiments.columns.str.endswith('_drop')]
sentiments

,date,custom_ref,reference,title,author,is_gov,text,finbert_pos,finbert_neg,finbert_neu,finbert_sentiment,finbert_score,finbert_confidence,central_bank,financial_sector,firms,government,households,cbroberta_avg_score,cbroberta_avg_sentiment
0,1998-09-15,UK_1998-09-15_George,r980915a_BOE,Speech,Edward George,1.0,"Thank you, Chairman. I'm actually very pleased...",0.081732,0.281070,0.637198,neutral,-0.199338,0.637198,-0.409745,-0.661564,-0.202167,-0.518037,-0.042013,-0.295018,0
1,1998-10-21,UK_1998-10-21_George,r981021b_BOE,Britain in Europe,Edward George,1.0,It's a great pleasure to be here in the beauti...,0.153660,0.028016,0.818323,neutral,0.125644,0.818323,-0.134019,0.047685,-0.305432,-0.251951,-0.535169,-0.128637,0
2,1998-10-21,UK_1998-10-21_King,r981021a_BOE,Impact of the recent turbulence in internation...,Mervyn King,0.0,Few industries have suffered more from volatil...,0.038727,0.808180,0.153093,negative,-0.769453,0.808180,-0.516879,-0.644094,-0.581463,-0.580222,-0.672911,-0.601670,0
3,1998-11-01,UK_1998-11-01_budd,r981101a_BOE,"Economic policy, with and without forecasts",budd,0.0,My topic this evening is the use of forecasts ...,0.062295,0.135857,0.801848,neutral,-0.073561,0.801848,-0.497668,-0.373081,-0.598642,-0.567226,-0.360106,-0.476216,0
4,1998-11-01,UK_1998-11-01_vickers,r981101b_BOE,Inflation targeting in practice: the UK experi...,vickers,0.0,"Six years ago this week, sterling left the exc...",0.082896,0.082629,0.834475,neutral,0.000268,0.834475,-0.555400,-0.629627,-0.683497,-0.339261,-0.289324,-0.569214,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1360,2025-02-28,UK_2025-02-28_Ramsden,NaN,"Surveys, forecasts and scenarios: setting UK m...",Dave Ramsden,0.0,Speech Thank you for the invitation to speak ...,0.087475,0.032670,0.879856,neutral,0.054805,0.879856,-0.587502,-0.648696,-0.595713,-0.572169,-0.436912,-0.547658,0
1361,2025-03-03,UK_2025-03-03_Kyei,NaN,"Accountants can influence growth, competitiven...",Afua Kyei,0.0,"Speech Good evening everyone, students and al...",0.051385,0.035248,0.913366,neutral,0.016137,0.913366,0.479018,0.512251,0.355660,0.497915,0.269962,0.421629,1
1362,2025-03-06,UK_2025-03-06_Mann,NaN,Holding the anchor in turbulent waters  speec...,L. Mann,0.0,Speech Introduction It is particularly fitti...,0.101165,0.022226,0.876609,neutral,0.078938,0.876609,-0.567989,-0.744008,-0.746682,-0.258689,-0.682128,-0.634426,0
1363,2025-03-24,UK_2025-03-24_Bailey,NaN,Growth: What does it take in todays world? - ...,Andrew Bailey,1.0,Speech Thank you for inviting me to speak tod...,0.059699,0.031643,0.908659,neutral,0.028056,0.908659,-0.159905,-0.521657,0.081224,-0.194019,-0.030078,-0.046111,0


## Join all daily indicators with sentiment scores

In [236]:
daily_indicators.dtypes

date         period[D]
ftse250        float64
ftse100        float64
gilt_2y        float64
gilt_10y       float64
ois_1m         float64
ois_3m         float64
ois_6m         float64
ois_12m        float64
sonia          float64
vix            float64
gbp_usd_d      float64
dtype: object

In [237]:
daily_indicators['date'] = daily_indicators['date'].dt.to_timestamp()

In [238]:
sentiments.dtypes

date                        object
custom_ref                  object
reference                   object
title                       object
author                      object
is_gov                     float64
text                        object
finbert_pos                float64
finbert_neg                float64
finbert_neu                float64
finbert_sentiment           object
finbert_score              float64
finbert_confidence         float64
central_bank               float64
financial_sector           float64
firms                      float64
government                 float64
households                 float64
cbroberta_avg_score        float64
cbroberta_avg_sentiment      int64
dtype: object

In [239]:
sentiments['date'] = pd.to_datetime(sentiments['date'])

In [240]:
# Perform an outer join
sentiments_daily_indicators_all = sentiments.merge(daily_indicators, on='date', how='outer')
sentiments_daily_indicators_all

,date,custom_ref,reference,title,author,is_gov,text,finbert_pos,finbert_neg,finbert_neu,...,ftse100,gilt_2y,gilt_10y,ois_1m,ois_3m,ois_6m,ois_12m,sonia,vix,gbp_usd_d
0,1998-09-15,UK_1998-09-15_George,r980915a_BOE,Speech,Edward George,1.0,"Thank you, Chairman. I'm actually very pleased...",0.081732,0.281070,0.637198,...,5281.73,NaN,NaN,NaN,NaN,NaN,NaN,7.6132,36.58,1.6742
1,1998-10-21,UK_1998-10-21_George,r981021b_BOE,Britain in Europe,Edward George,1.0,It's a great pleasure to be here in the beauti...,0.153660,0.028016,0.818323,...,5206.65,NaN,NaN,NaN,NaN,NaN,NaN,7.4073,33.21,1.7005
2,1998-10-21,UK_1998-10-21_King,r981021a_BOE,Impact of the recent turbulence in internation...,Mervyn King,0.0,Few industries have suffered more from volatil...,0.038727,0.808180,0.153093,...,5206.65,NaN,NaN,NaN,NaN,NaN,NaN,7.4073,33.21,1.7005
3,1998-11-01,UK_1998-11-01_budd,r981101a_BOE,"Economic policy, with and without forecasts",budd,0.0,My topic this evening is the use of forecasts ...,0.062295,0.135857,0.801848,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1998-11-01,UK_1998-11-01_vickers,r981101b_BOE,Inflation targeting in practice: the UK experi...,vickers,0.0,"Six years ago this week, sterling left the exc...",0.082896,0.082629,0.834475,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10391,2025-05-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.64,1.3284
10392,2025-05-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.76,1.3369
10393,2025-05-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.55,1.3344
10394,2025-05-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.3287


In [241]:
filtered_sentiments_daily_indicators_all = sentiments_daily_indicators_all[sentiments_daily_indicators_all.date.between('2014-10-01', '2024-12-31')]
filtered_sentiments_daily_indicators_all

,date,custom_ref,reference,title,author,is_gov,text,finbert_pos,finbert_neg,finbert_neu,...,ftse100,gilt_2y,gilt_10y,ois_1m,ois_3m,ois_6m,ois_12m,sonia,vix,gbp_usd_d
658,2014-10-01,UK_2014-10-01_fisher,r141001a_BOE,"Microprudential, macroprudential and monetary ...",fisher,0.0,Thank you for inviting me once again to speak ...,0.086988,0.018759,0.894253,...,6557.52000,0.88,2.41,NaN,NaN,NaN,NaN,0.4285,16.71,1.6216
659,2014-10-01,UK_2014-10-01_forbes,r141001b_BOE,The economic impact of sterling's recent moves...,forbes,0.0,Last summer I had the pleasure of seeing my fi...,0.086558,0.067932,0.845510,...,6557.52000,0.88,2.41,NaN,NaN,NaN,NaN,0.4285,16.71,1.6216
660,2014-10-12,UK_2014-10-12_Carney,r141012a_BOE,Regulatory work underway and lessons learned,Mark Carney,1.0,Six years ago when the financial crisis was in...,0.116043,0.089111,0.794846,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
661,2014-10-15,UK_2014-10-15_weale,r141015a_BOE,What to do when we don't know: policy-making w...,weale,0.0,Could I say what a great privilege and pleasur...,0.044438,0.128805,0.826757,...,6211.64000,0.59,2.04,NaN,NaN,NaN,NaN,0.4290,26.25,1.5959
662,2014-10-16,UK_2014-10-16_no_info,r141016a_BOE,Andrew Bailey's speech at the Lord Mayor's Ban...,no_info,0.0,"Lord Mayor, thank you once again for inviting ...",0.120008,0.031809,0.848183,...,6195.91000,0.69,2.12,NaN,NaN,NaN,NaN,0.4314,25.20,1.6065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10313,2024-12-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10314,2024-12-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.73,1.2522
10315,2024-12-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8149.78040,4.26,4.62,4.69,4.61,4.53,4.40,4.7000,15.95,1.2573
10316,2024-12-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8121.00768,4.23,4.60,4.69,4.61,4.52,4.39,4.7000,17.40,1.2523


In [242]:
# Save to csv
filtered_sentiments_daily_indicators_all.to_csv('sentiments_daily_indicators_all.csv', index=False)

## Join all monthly indicators with sentiment scores

In [243]:
# Add month column to sentiments dataframe
sentiments['month'] = sentiments['date'].dt.to_period('M')
sentiments

,date,custom_ref,reference,title,author,is_gov,text,finbert_pos,finbert_neg,finbert_neu,...,finbert_score,finbert_confidence,central_bank,financial_sector,firms,government,households,cbroberta_avg_score,cbroberta_avg_sentiment,month
0,1998-09-15,UK_1998-09-15_George,r980915a_BOE,Speech,Edward George,1.0,"Thank you, Chairman. I'm actually very pleased...",0.081732,0.281070,0.637198,...,-0.199338,0.637198,-0.409745,-0.661564,-0.202167,-0.518037,-0.042013,-0.295018,0,1998-09
1,1998-10-21,UK_1998-10-21_George,r981021b_BOE,Britain in Europe,Edward George,1.0,It's a great pleasure to be here in the beauti...,0.153660,0.028016,0.818323,...,0.125644,0.818323,-0.134019,0.047685,-0.305432,-0.251951,-0.535169,-0.128637,0,1998-10
2,1998-10-21,UK_1998-10-21_King,r981021a_BOE,Impact of the recent turbulence in internation...,Mervyn King,0.0,Few industries have suffered more from volatil...,0.038727,0.808180,0.153093,...,-0.769453,0.808180,-0.516879,-0.644094,-0.581463,-0.580222,-0.672911,-0.601670,0,1998-10
3,1998-11-01,UK_1998-11-01_budd,r981101a_BOE,"Economic policy, with and without forecasts",budd,0.0,My topic this evening is the use of forecasts ...,0.062295,0.135857,0.801848,...,-0.073561,0.801848,-0.497668,-0.373081,-0.598642,-0.567226,-0.360106,-0.476216,0,1998-11
4,1998-11-01,UK_1998-11-01_vickers,r981101b_BOE,Inflation targeting in practice: the UK experi...,vickers,0.0,"Six years ago this week, sterling left the exc...",0.082896,0.082629,0.834475,...,0.000268,0.834475,-0.555400,-0.629627,-0.683497,-0.339261,-0.289324,-0.569214,0,1998-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1360,2025-02-28,UK_2025-02-28_Ramsden,NaN,"Surveys, forecasts and scenarios: setting UK m...",Dave Ramsden,0.0,Speech Thank you for the invitation to speak ...,0.087475,0.032670,0.879856,...,0.054805,0.879856,-0.587502,-0.648696,-0.595713,-0.572169,-0.436912,-0.547658,0,2025-02
1361,2025-03-03,UK_2025-03-03_Kyei,NaN,"Accountants can influence growth, competitiven...",Afua Kyei,0.0,"Speech Good evening everyone, students and al...",0.051385,0.035248,0.913366,...,0.016137,0.913366,0.479018,0.512251,0.355660,0.497915,0.269962,0.421629,1,2025-03
1362,2025-03-06,UK_2025-03-06_Mann,NaN,Holding the anchor in turbulent waters  speec...,L. Mann,0.0,Speech Introduction It is particularly fitti...,0.101165,0.022226,0.876609,...,0.078938,0.876609,-0.567989,-0.744008,-0.746682,-0.258689,-0.682128,-0.634426,0,2025-03
1363,2025-03-24,UK_2025-03-24_Bailey,NaN,Growth: What does it take in todays world? - ...,Andrew Bailey,1.0,Speech Thank you for inviting me to speak tod...,0.059699,0.031643,0.908659,...,0.028056,0.908659,-0.159905,-0.521657,0.081224,-0.194019,-0.030078,-0.046111,0,2025-03


In [244]:
# Perform an outer join
sentiments_monthly_indicators_all = sentiments.merge(monthly_indicators, on='month', how='outer')
sentiments_monthly_indicators_all

,date,custom_ref,reference,title,author,is_gov,text,finbert_pos,finbert_neg,finbert_neu,...,wage_rate,pmi,ftse100_monthly,ftse100_volat,ftse250_monthly,ftse250_volat,ftse_all_share_monthly,ftse_all_share_volat,gdp_rate,household_debt
0,1998-09-15,UK_1998-09-15_George,r980915a_BOE,Speech,Edward George,1.0,"Thank you, Chairman. I'm actually very pleased...",0.081732,0.281070,0.637198,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.3,92.6
1,1998-10-21,UK_1998-10-21_George,r981021b_BOE,Britain in Europe,Edward George,1.0,It's a great pleasure to be here in the beauti...,0.153660,0.028016,0.818323,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.7,93.3
2,1998-10-21,UK_1998-10-21_King,r981021a_BOE,Impact of the recent turbulence in internation...,Mervyn King,0.0,Few industries have suffered more from volatil...,0.038727,0.808180,0.153093,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.7,93.3
3,1998-11-01,UK_1998-11-01_budd,r981101a_BOE,"Economic policy, with and without forecasts",budd,0.0,My topic this evening is the use of forecasts ...,0.062295,0.135857,0.801848,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.7,93.3
4,1998-11-01,UK_1998-11-01_vickers,r981101b_BOE,Inflation targeting in practice: the UK experi...,vickers,0.0,"Six years ago this week, sterling left the exc...",0.082896,0.082629,0.834475,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.7,93.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1902,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.0,57.25,7119.7002,24.569728,24102.19,25.929313,"4,109.96",24.471628,1.7,137.3
1903,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.5,51.50,7284.1499,20.421172,19063.75,34.463209,"4,007.46",21.760138,0.1,135.5
1904,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.5,52.35,8164.1200,17.216341,20286.03,31.101506,"4,451.92",18.616010,0.5,121.0
1905,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,49.85,8494.8500,17.983591,19884.59,22.063025,"4,594.05",17.145201,NaN,NaN


In [245]:
# Assuming df is your dataframe and 'date_col' is your Period column
start_date = pd.Period('2014-01-01', freq='M')
end_date = pd.Period('2025-04-30', freq='M')

# Filter using comparison operators
filtered_sentiments_monthly_indicators_all = sentiments_monthly_indicators_all[(sentiments_monthly_indicators_all['month'] >= start_date) & (sentiments_monthly_indicators_all['month'] <= end_date)]

filtered_sentiments_monthly_indicators_all

,date,custom_ref,reference,title,author,is_gov,text,finbert_pos,finbert_neg,finbert_neu,...,wage_rate,pmi,ftse100_monthly,ftse100_volat,ftse250_monthly,ftse250_volat,ftse_all_share_monthly,ftse_all_share_volat,gdp_rate,household_debt
613,2014-01-17,UK_2014-01-17_broadbent,r140117a_BOE,The balance of growth,broadbent,0.0,"The UK is economy is growing, and will probabl...",0.071130,0.706857,0.222014,...,1.1,NaN,6510.4399,25.261675,NaN,22.606845,"3,496.51",24.602883,0.8,138.8
614,2014-01-22,UK_2014-01-22_mccafferty,r140122a_BOE,Achieving a sustainable recovery: where next f...,mccafferty,0.0,"This morning, the ONS released its latest Labo...",0.259893,0.360966,0.379141,...,1.1,NaN,6510.4399,25.261675,NaN,22.606845,"3,496.51",24.602883,0.8,138.8
615,2014-01-23,UK_2014-01-23_fisher,r140123a_BOE,"Inflation, interest rates and forward guidance",fisher,0.0,This morning I would like to discuss some of t...,0.049391,0.557097,0.393512,...,1.1,NaN,6510.4399,25.261675,NaN,22.606845,"3,496.51",24.602883,0.8,138.8
616,2014-01-24,UK_2014-01-24_Carney,r140124a_BOE,Remarks given,Mark Carney,1.0,When a banker invites a regulator to a meeting...,0.420347,0.175880,0.403773,...,1.1,NaN,6510.4399,25.261675,NaN,22.606845,"3,496.51",24.602883,0.8,138.8
617,2014-01-29,UK_2014-01-29_industry,r140129a_BOE,Mark Carney: Speech at lunch hosted,industry,0.0,It is a pleasure to be in Scotland today and t...,0.178864,0.068882,0.752254,...,1.1,NaN,6510.4399,25.261675,NaN,22.606845,"3,496.51",24.602883,0.8,138.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1901,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.3,49.25,7346.5298,20.647015,20812.60,17.682857,"4,066.73",19.152907,0.0,134.9
1902,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.0,57.25,7119.7002,24.569728,24102.19,25.929313,"4,109.96",24.471628,1.7,137.3
1903,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.5,51.50,7284.1499,20.421172,19063.75,34.463209,"4,007.46",21.760138,0.1,135.5
1904,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.5,52.35,8164.1200,17.216341,20286.03,31.101506,"4,451.92",18.616010,0.5,121.0


In [246]:
# Save to csv
filtered_sentiments_monthly_indicators_all.to_csv('sentiments_monthly_indicators_all.csv', index=False)

## Join all quarterly indicators with sentiment scores

In [247]:
# Add month column to sentiments dataframe
sentiments['quarter'] = sentiments['date'].dt.to_period('Q')
sentiments

,date,custom_ref,reference,title,author,is_gov,text,finbert_pos,finbert_neg,finbert_neu,...,finbert_confidence,central_bank,financial_sector,firms,government,households,cbroberta_avg_score,cbroberta_avg_sentiment,month,quarter
0,1998-09-15,UK_1998-09-15_George,r980915a_BOE,Speech,Edward George,1.0,"Thank you, Chairman. I'm actually very pleased...",0.081732,0.281070,0.637198,...,0.637198,-0.409745,-0.661564,-0.202167,-0.518037,-0.042013,-0.295018,0,1998-09,1998Q3
1,1998-10-21,UK_1998-10-21_George,r981021b_BOE,Britain in Europe,Edward George,1.0,It's a great pleasure to be here in the beauti...,0.153660,0.028016,0.818323,...,0.818323,-0.134019,0.047685,-0.305432,-0.251951,-0.535169,-0.128637,0,1998-10,1998Q4
2,1998-10-21,UK_1998-10-21_King,r981021a_BOE,Impact of the recent turbulence in internation...,Mervyn King,0.0,Few industries have suffered more from volatil...,0.038727,0.808180,0.153093,...,0.808180,-0.516879,-0.644094,-0.581463,-0.580222,-0.672911,-0.601670,0,1998-10,1998Q4
3,1998-11-01,UK_1998-11-01_budd,r981101a_BOE,"Economic policy, with and without forecasts",budd,0.0,My topic this evening is the use of forecasts ...,0.062295,0.135857,0.801848,...,0.801848,-0.497668,-0.373081,-0.598642,-0.567226,-0.360106,-0.476216,0,1998-11,1998Q4
4,1998-11-01,UK_1998-11-01_vickers,r981101b_BOE,Inflation targeting in practice: the UK experi...,vickers,0.0,"Six years ago this week, sterling left the exc...",0.082896,0.082629,0.834475,...,0.834475,-0.555400,-0.629627,-0.683497,-0.339261,-0.289324,-0.569214,0,1998-11,1998Q4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1360,2025-02-28,UK_2025-02-28_Ramsden,NaN,"Surveys, forecasts and scenarios: setting UK m...",Dave Ramsden,0.0,Speech Thank you for the invitation to speak ...,0.087475,0.032670,0.879856,...,0.879856,-0.587502,-0.648696,-0.595713,-0.572169,-0.436912,-0.547658,0,2025-02,2025Q1
1361,2025-03-03,UK_2025-03-03_Kyei,NaN,"Accountants can influence growth, competitiven...",Afua Kyei,0.0,"Speech Good evening everyone, students and al...",0.051385,0.035248,0.913366,...,0.913366,0.479018,0.512251,0.355660,0.497915,0.269962,0.421629,1,2025-03,2025Q1
1362,2025-03-06,UK_2025-03-06_Mann,NaN,Holding the anchor in turbulent waters  speec...,L. Mann,0.0,Speech Introduction It is particularly fitti...,0.101165,0.022226,0.876609,...,0.876609,-0.567989,-0.744008,-0.746682,-0.258689,-0.682128,-0.634426,0,2025-03,2025Q1
1363,2025-03-24,UK_2025-03-24_Bailey,NaN,Growth: What does it take in todays world? - ...,Andrew Bailey,1.0,Speech Thank you for inviting me to speak tod...,0.059699,0.031643,0.908659,...,0.908659,-0.159905,-0.521657,0.081224,-0.194019,-0.030078,-0.046111,0,2025-03,2025Q1


In [248]:
# Perform an outer join
sentiments_quarterly_indicators_all = sentiments.merge(quarterly_indicators, on='quarter', how='outer')
sentiments_quarterly_indicators_all

,date,custom_ref,reference,title,author,is_gov,text,finbert_pos,finbert_neg,finbert_neu,...,loans_to_pvt_sector,unemp_rate,business_confidence,consumer_spending,wage_rate,rsi_3mo3m,rsi_vol,house_price_change,gfk_avg_cc_qrt,infl_exp_qrt
0,1998-09-15,UK_1998-09-15_George,r980915a_BOE,Speech,Edward George,1.0,"Thank you, Chairman. I'm actually very pleased...",0.081732,0.281070,0.637198,...,891729.0,6.2,-44.0,250494.0,NaN,0.7,62.3,1.31,-1.33,NaN
1,1998-10-21,UK_1998-10-21_George,r981021b_BOE,Britain in Europe,Edward George,1.0,It's a great pleasure to be here in the beauti...,0.153660,0.028016,0.818323,...,904422.0,6.1,-58.0,250464.0,NaN,-0.5,62.0,0.70,-6.33,NaN
2,1998-10-21,UK_1998-10-21_King,r981021a_BOE,Impact of the recent turbulence in internation...,Mervyn King,0.0,Few industries have suffered more from volatil...,0.038727,0.808180,0.153093,...,904422.0,6.1,-58.0,250464.0,NaN,-0.5,62.0,0.70,-6.33,NaN
3,1998-11-01,UK_1998-11-01_budd,r981101a_BOE,"Economic policy, with and without forecasts",budd,0.0,My topic this evening is the use of forecasts ...,0.062295,0.135857,0.801848,...,904422.0,6.1,-58.0,250464.0,NaN,-0.5,62.0,0.70,-6.33,NaN
4,1998-11-01,UK_1998-11-01_vickers,r981101b_BOE,Inflation targeting in practice: the UK experi...,vickers,0.0,"Six years ago this week, sterling left the exc...",0.082896,0.082629,0.834475,...,904422.0,6.1,-58.0,250464.0,NaN,-0.5,62.0,0.70,-6.33,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1535,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,829385.0,6.8,-6.0,239345.0,NaN,-0.2,60.9,1.81,NaN,NaN
1536,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,845524.0,6.5,2.0,241963.0,NaN,1.6,61.8,1.51,NaN,NaN
1537,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,865108.0,6.4,-11.0,243935.0,NaN,0.2,61.9,1.81,NaN,NaN
1538,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,876419.0,6.3,-22.0,246304.0,NaN,-0.1,61.9,1.61,4.33,NaN


In [249]:
# Assuming df is your dataframe and 'date_col' is your Period column
start_date = pd.Period('2014-01-01', freq='Q')
end_date = pd.Period('2025-04-30', freq='Q')

# Filter using comparison operators
filtered_sentiments_quarterly_indicators_all = sentiments_quarterly_indicators_all[(sentiments_quarterly_indicators_all['quarter'] >= start_date) & (sentiments_quarterly_indicators_all['quarter'] <= end_date)]

filtered_sentiments_quarterly_indicators_all

,date,custom_ref,reference,title,author,is_gov,text,finbert_pos,finbert_neg,finbert_neu,...,loans_to_pvt_sector,unemp_rate,business_confidence,consumer_spending,wage_rate,rsi_3mo3m,rsi_vol,house_price_change,gfk_avg_cc_qrt,infl_exp_qrt
613,2014-01-17,UK_2014-01-17_broadbent,r140117a_BOE,The balance of growth,broadbent,0.0,"The UK is economy is growing, and will probabl...",0.071130,0.706857,0.222014,...,2184495.0,6.7,21.0,339058.0,1.17,1.3,85.1,2.52,-6.33,2.23
614,2014-01-22,UK_2014-01-22_mccafferty,r140122a_BOE,Achieving a sustainable recovery: where next f...,mccafferty,0.0,"This morning, the ONS released its latest Labo...",0.259893,0.360966,0.379141,...,2184495.0,6.7,21.0,339058.0,1.17,1.3,85.1,2.52,-6.33,2.23
615,2014-01-23,UK_2014-01-23_fisher,r140123a_BOE,"Inflation, interest rates and forward guidance",fisher,0.0,This morning I would like to discuss some of t...,0.049391,0.557097,0.393512,...,2184495.0,6.7,21.0,339058.0,1.17,1.3,85.1,2.52,-6.33,2.23
616,2014-01-24,UK_2014-01-24_Carney,r140124a_BOE,Remarks given,Mark Carney,1.0,When a banker invites a regulator to a meeting...,0.420347,0.175880,0.403773,...,2184495.0,6.7,21.0,339058.0,1.17,1.3,85.1,2.52,-6.33,2.23
617,2014-01-29,UK_2014-01-29_industry,r140129a_BOE,Mark Carney: Speech at lunch hosted,industry,0.0,It is a pleasure to be in Scotland today and t...,0.178864,0.068882,0.752254,...,2184495.0,6.7,21.0,339058.0,1.17,1.3,85.1,2.52,-6.33,2.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1361,2025-03-03,UK_2025-03-03_Kyei,NaN,"Accountants can influence growth, competitiven...",Afua Kyei,0.0,"Speech Good evening everyone, students and al...",0.051385,0.035248,0.913366,...,NaN,NaN,-47.0,NaN,5.85,1.6,99.2,-0.10,-20.33,NaN
1362,2025-03-06,UK_2025-03-06_Mann,NaN,Holding the anchor in turbulent waters  speec...,L. Mann,0.0,Speech Introduction It is particularly fitti...,0.101165,0.022226,0.876609,...,NaN,NaN,-47.0,NaN,5.85,1.6,99.2,-0.10,-20.33,NaN
1363,2025-03-24,UK_2025-03-24_Bailey,NaN,Growth: What does it take in todays world? - ...,Andrew Bailey,1.0,Speech Thank you for inviting me to speak tod...,0.059699,0.031643,0.908659,...,NaN,NaN,-47.0,NaN,5.85,1.6,99.2,-0.10,-20.33,NaN
1364,2025-03-25,UK_2025-03-25_Cleland,NaN,Innovating wholesale payments: building a resi...,Victoria Cleland,0.0,Speech Introduction Over the last quarter of...,0.523743,0.010683,0.465575,...,NaN,NaN,-47.0,NaN,5.85,1.6,99.2,-0.10,-20.33,NaN


In [250]:
# Save to csv
filtered_sentiments_quarterly_indicators_all.to_csv('sentiments_quarterly_indicators_all.csv', index=False)

## Add topic scores to sentiment with daily indicators

In [251]:
# Import .csv file
topics = pd.read_csv('uk_data_with_topics.csv')
topics

,Unnamed: 0,custom_ref,reference,country,date,title,author,is_gov,text,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,dominant_topic,dominant_topic_name
0,0,UK_1998-09-15_George,r980915a_BOE,UK,1998-09-15,Speech,Edward George,1.0,"Thank you, Chairman. I'm actually very pleased...",0.019365,0.000115,0.000115,0.000115,0.016494,0.000115,0.000115,0.963565,7,Inflation & Monetary Policy
1,1,UK_1998-10-21_George,r981021b_BOE,UK,1998-10-21,Britain in Europe,Edward George,1.0,It's a great pleasure to be here in the beauti...,0.397453,0.031454,0.000063,0.000063,0.000063,0.000063,0.000063,0.570779,7,Inflation & Monetary Policy
2,2,UK_1998-10-21_King,r981021a_BOE,UK,1998-10-21,Impact of the recent turbulence in internation...,Mervyn King,0.0,Few industries have suffered more from volatil...,0.000108,0.209397,0.000108,0.000108,0.000108,0.044162,0.028020,0.717988,7,Inflation & Monetary Policy
3,3,UK_1998-11-01_budd,r981101a_BOE,UK,1998-11-01,"Economic policy, with and without forecasts",budd,0.0,My topic this evening is the use of forecasts ...,0.113619,0.000056,0.000056,0.377057,0.186295,0.000056,0.037116,0.285746,3,Forecasting & Uncertainty Modelling
4,4,UK_1998-11-01_vickers,r981101b_BOE,UK,1998-11-01,Inflation targeting in practice: the UK experi...,vickers,0.0,"Six years ago this week, sterling left the exc...",0.039526,0.000053,0.000053,0.245717,0.018334,0.000053,0.041993,0.654271,7,Inflation & Monetary Policy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1360,1360,UK_2025-02-28_Ramsden,NaN,UK,2025-02-28,"Surveys, forecasts and scenarios: setting UK m...",Dave Ramsden,0.0,Speech Thank you for the invitation to speak ...,0.000045,0.000045,0.000045,0.507005,0.000045,0.000045,0.075986,0.416784,3,Forecasting & Uncertainty Modelling
1361,1361,UK_2025-03-03_Kyei,NaN,UK,2025-03-03,"Accountants can influence growth, competitiven...",Afua Kyei,0.0,"Speech Good evening everyone, students and al...",0.044976,0.000187,0.000187,0.067141,0.834656,0.000187,0.052479,0.000187,4,Banking & Insurance Supervision
1362,1362,UK_2025-03-06_Mann,NaN,UK,2025-03-06,Holding the anchor in turbulent waters  speec...,L. Mann,0.0,Speech Introduction It is particularly fitti...,0.000045,0.000045,0.000045,0.397302,0.000045,0.000045,0.000045,0.602429,7,Inflation & Monetary Policy
1363,1363,UK_2025-03-24_Bailey,NaN,UK,2025-03-24,Growth: What does it take in todays world? - ...,Andrew Bailey,1.0,Speech Thank you for inviting me to speak tod...,0.040196,0.000047,0.000047,0.094383,0.053026,0.000047,0.615757,0.196498,6,Labour Markets & Productivity


In [252]:
# Select required columns
topics = topics[['custom_ref', 'topic_0', 'topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5', 'topic_6', 'topic_7',	'dominant_topic', 'dominant_topic_name']]
topics

,custom_ref,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,dominant_topic,dominant_topic_name
0,UK_1998-09-15_George,0.019365,0.000115,0.000115,0.000115,0.016494,0.000115,0.000115,0.963565,7,Inflation & Monetary Policy
1,UK_1998-10-21_George,0.397453,0.031454,0.000063,0.000063,0.000063,0.000063,0.000063,0.570779,7,Inflation & Monetary Policy
2,UK_1998-10-21_King,0.000108,0.209397,0.000108,0.000108,0.000108,0.044162,0.028020,0.717988,7,Inflation & Monetary Policy
3,UK_1998-11-01_budd,0.113619,0.000056,0.000056,0.377057,0.186295,0.000056,0.037116,0.285746,3,Forecasting & Uncertainty Modelling
4,UK_1998-11-01_vickers,0.039526,0.000053,0.000053,0.245717,0.018334,0.000053,0.041993,0.654271,7,Inflation & Monetary Policy
...,...,...,...,...,...,...,...,...,...,...,...
1360,UK_2025-02-28_Ramsden,0.000045,0.000045,0.000045,0.507005,0.000045,0.000045,0.075986,0.416784,3,Forecasting & Uncertainty Modelling
1361,UK_2025-03-03_Kyei,0.044976,0.000187,0.000187,0.067141,0.834656,0.000187,0.052479,0.000187,4,Banking & Insurance Supervision
1362,UK_2025-03-06_Mann,0.000045,0.000045,0.000045,0.397302,0.000045,0.000045,0.000045,0.602429,7,Inflation & Monetary Policy
1363,UK_2025-03-24_Bailey,0.040196,0.000047,0.000047,0.094383,0.053026,0.000047,0.615757,0.196498,6,Labour Markets & Productivity


In [253]:
# Perform a left join
sentiments_topics_daily_indicators = filtered_sentiments_daily_indicators_all.merge(topics, on='custom_ref', how='left')
sentiments_topics_daily_indicators

,date,custom_ref,reference,title,author,is_gov,text,finbert_pos,finbert_neg,finbert_neu,...,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,dominant_topic,dominant_topic_name
0,2014-10-01,UK_2014-10-01_fisher,r141001a_BOE,"Microprudential, macroprudential and monetary ...",fisher,0.0,Thank you for inviting me once again to speak ...,0.086988,0.018759,0.894253,...,0.667635,0.042078,0.000055,0.000055,0.155470,0.018992,0.000055,0.115658,0.0,Financial Stability & Governance
1,2014-10-01,UK_2014-10-01_forbes,r141001b_BOE,The economic impact of sterling's recent moves...,forbes,0.0,Last summer I had the pleasure of seeing my fi...,0.086558,0.067932,0.845510,...,0.000045,0.000045,0.000045,0.056788,0.000045,0.000045,0.370498,0.572490,7.0,Inflation & Monetary Policy
2,2014-10-12,UK_2014-10-12_Carney,r141012a_BOE,Regulatory work underway and lessons learned,Mark Carney,1.0,Six years ago when the financial crisis was in...,0.116043,0.089111,0.794846,...,0.164216,0.088025,0.000116,0.000116,0.666952,0.080344,0.000116,0.000116,4.0,Banking & Insurance Supervision
3,2014-10-15,UK_2014-10-15_weale,r141015a_BOE,What to do when we don't know: policy-making w...,weale,0.0,Could I say what a great privilege and pleasur...,0.044438,0.128805,0.826757,...,0.000046,0.000046,0.000046,0.281561,0.000046,0.000046,0.063231,0.654979,7.0,Inflation & Monetary Policy
4,2014-10-16,UK_2014-10-16_no_info,r141016a_BOE,Andrew Bailey's speech at the Lord Mayor's Ban...,no_info,0.0,"Lord Mayor, thank you once again for inviting ...",0.120008,0.031809,0.848183,...,0.000094,0.000094,0.000094,0.000094,0.999342,0.000094,0.000094,0.000094,4.0,Banking & Insurance Supervision
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2772,2024-12-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2773,2024-12-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2774,2024-12-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2775,2024-12-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [254]:
# Save to csv
sentiments_topics_daily_indicators.to_csv('sentiments_topics_daily_indicators.csv', index=False)

## Add topic scores to sentiment with monthly indicators

In [255]:
# Perform a left join
sentiments_topics_monthly_indicators = filtered_sentiments_monthly_indicators_all.merge(topics, on='custom_ref', how='left')
sentiments_topics_monthly_indicators

,date,custom_ref,reference,title,author,is_gov,text,finbert_pos,finbert_neg,finbert_neu,...,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,dominant_topic,dominant_topic_name
0,2014-01-17,UK_2014-01-17_broadbent,r140117a_BOE,The balance of growth,broadbent,0.0,"The UK is economy is growing, and will probabl...",0.071130,0.706857,0.222014,...,0.000106,0.000106,0.000105,0.000106,0.000106,0.000106,0.169998,0.829368,7.0,Inflation & Monetary Policy
1,2014-01-22,UK_2014-01-22_mccafferty,r140122a_BOE,Achieving a sustainable recovery: where next f...,mccafferty,0.0,"This morning, the ONS released its latest Labo...",0.259893,0.360966,0.379141,...,0.000055,0.429016,0.000055,0.000055,0.000055,0.039322,0.311531,0.219911,1.0,Credit & Housing Markets
2,2014-01-23,UK_2014-01-23_fisher,r140123a_BOE,"Inflation, interest rates and forward guidance",fisher,0.0,This morning I would like to discuss some of t...,0.049391,0.557097,0.393512,...,0.000078,0.104059,0.000078,0.000078,0.000078,0.000078,0.000078,0.895471,7.0,Inflation & Monetary Policy
3,2014-01-24,UK_2014-01-24_Carney,r140124a_BOE,Remarks given,Mark Carney,1.0,When a banker invites a regulator to a meeting...,0.420347,0.175880,0.403773,...,0.055844,0.110221,0.000072,0.000072,0.202224,0.000072,0.073162,0.558333,7.0,Inflation & Monetary Policy
4,2014-01-29,UK_2014-01-29_industry,r140129a_BOE,Mark Carney: Speech at lunch hosted,industry,0.0,It is a pleasure to be in Scotland today and t...,0.178864,0.068882,0.752254,...,0.304140,0.197406,0.000063,0.000063,0.154984,0.000063,0.094691,0.248590,0.0,Financial Stability & Governance
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
755,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
756,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
757,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [256]:
# Save to csv
sentiments_topics_monthly_indicators.to_csv('sentiments_topics_monthly_indicators.csv', index=False)

## Add topic scores to sentiment with quarterly indicators

In [257]:
# Perform a left join
sentiments_topics_quarterly_indicators = filtered_sentiments_quarterly_indicators_all.merge(topics, on='custom_ref', how='left')
sentiments_topics_quarterly_indicators

,date,custom_ref,reference,title,author,is_gov,text,finbert_pos,finbert_neg,finbert_neu,...,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,dominant_topic,dominant_topic_name
0,2014-01-17,UK_2014-01-17_broadbent,r140117a_BOE,The balance of growth,broadbent,0.0,"The UK is economy is growing, and will probabl...",0.071130,0.706857,0.222014,...,0.000106,0.000106,0.000105,0.000106,0.000106,0.000106,0.169998,0.829368,7.0,Inflation & Monetary Policy
1,2014-01-22,UK_2014-01-22_mccafferty,r140122a_BOE,Achieving a sustainable recovery: where next f...,mccafferty,0.0,"This morning, the ONS released its latest Labo...",0.259893,0.360966,0.379141,...,0.000055,0.429016,0.000055,0.000055,0.000055,0.039322,0.311531,0.219911,1.0,Credit & Housing Markets
2,2014-01-23,UK_2014-01-23_fisher,r140123a_BOE,"Inflation, interest rates and forward guidance",fisher,0.0,This morning I would like to discuss some of t...,0.049391,0.557097,0.393512,...,0.000078,0.104059,0.000078,0.000078,0.000078,0.000078,0.000078,0.895471,7.0,Inflation & Monetary Policy
3,2014-01-24,UK_2014-01-24_Carney,r140124a_BOE,Remarks given,Mark Carney,1.0,When a banker invites a regulator to a meeting...,0.420347,0.175880,0.403773,...,0.055844,0.110221,0.000072,0.000072,0.202224,0.000072,0.073162,0.558333,7.0,Inflation & Monetary Policy
4,2014-01-29,UK_2014-01-29_industry,r140129a_BOE,Mark Carney: Speech at lunch hosted,industry,0.0,It is a pleasure to be in Scotland today and t...,0.178864,0.068882,0.752254,...,0.304140,0.197406,0.000063,0.000063,0.154984,0.000063,0.094691,0.248590,0.0,Financial Stability & Governance
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
748,2025-03-03,UK_2025-03-03_Kyei,NaN,"Accountants can influence growth, competitiven...",Afua Kyei,0.0,"Speech Good evening everyone, students and al...",0.051385,0.035248,0.913366,...,0.044976,0.000187,0.000187,0.067141,0.834656,0.000187,0.052479,0.000187,4.0,Banking & Insurance Supervision
749,2025-03-06,UK_2025-03-06_Mann,NaN,Holding the anchor in turbulent waters  speec...,L. Mann,0.0,Speech Introduction It is particularly fitti...,0.101165,0.022226,0.876609,...,0.000045,0.000045,0.000045,0.397302,0.000045,0.000045,0.000045,0.602429,7.0,Inflation & Monetary Policy
750,2025-03-24,UK_2025-03-24_Bailey,NaN,Growth: What does it take in todays world? - ...,Andrew Bailey,1.0,Speech Thank you for inviting me to speak tod...,0.059699,0.031643,0.908659,...,0.040196,0.000047,0.000047,0.094383,0.053026,0.000047,0.615757,0.196498,6.0,Labour Markets & Productivity
751,2025-03-25,UK_2025-03-25_Cleland,NaN,Innovating wholesale payments: building a resi...,Victoria Cleland,0.0,Speech Introduction Over the last quarter of...,0.523743,0.010683,0.465575,...,0.806499,0.000067,0.000067,0.000067,0.193100,0.000067,0.000067,0.000067,0.0,Financial Stability & Governance


In [258]:
# Save to csv
sentiments_topics_quarterly_indicators.to_csv('sentiments_topics_quarterly_indicators.csv', index=False)

### Add Bank Rates to sentiment with monthly indicators

In [259]:
monthly_all = sentiments_topics_monthly_indicators.merge(bank_rates_daily, on='date', how='left')
monthly_all

,date,custom_ref,reference,title,author,is_gov,text,finbert_pos,finbert_neg,finbert_neu,...,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,dominant_topic,dominant_topic_name,bank_rate
0,2014-01-17,UK_2014-01-17_broadbent,r140117a_BOE,The balance of growth,broadbent,0.0,"The UK is economy is growing, and will probabl...",0.071130,0.706857,0.222014,...,0.000106,0.000105,0.000106,0.000106,0.000106,0.169998,0.829368,7.0,Inflation & Monetary Policy,0.5
1,2014-01-22,UK_2014-01-22_mccafferty,r140122a_BOE,Achieving a sustainable recovery: where next f...,mccafferty,0.0,"This morning, the ONS released its latest Labo...",0.259893,0.360966,0.379141,...,0.429016,0.000055,0.000055,0.000055,0.039322,0.311531,0.219911,1.0,Credit & Housing Markets,0.5
2,2014-01-23,UK_2014-01-23_fisher,r140123a_BOE,"Inflation, interest rates and forward guidance",fisher,0.0,This morning I would like to discuss some of t...,0.049391,0.557097,0.393512,...,0.104059,0.000078,0.000078,0.000078,0.000078,0.000078,0.895471,7.0,Inflation & Monetary Policy,0.5
3,2014-01-24,UK_2014-01-24_Carney,r140124a_BOE,Remarks given,Mark Carney,1.0,When a banker invites a regulator to a meeting...,0.420347,0.175880,0.403773,...,0.110221,0.000072,0.000072,0.202224,0.000072,0.073162,0.558333,7.0,Inflation & Monetary Policy,0.5
4,2014-01-29,UK_2014-01-29_industry,r140129a_BOE,Mark Carney: Speech at lunch hosted,industry,0.0,It is a pleasure to be in Scotland today and t...,0.178864,0.068882,0.752254,...,0.197406,0.000063,0.000063,0.154984,0.000063,0.094691,0.248590,0.0,Financial Stability & Governance,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
755,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
756,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
757,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [260]:
# Save to csv
monthly_all.to_csv('monthly_all.csv', index=False)